In [44]:
#!pip install -q fast-ml

In [45]:
import os
os.chdir(os.path.dirname('/Users/danielmlow/Dropbox (MIT)/datum/ctl/'))
import pickle 
from collections import defaultdict
import re
import pandas as pd
import numpy as np
import sys
import pickle
# sys.path.insert(1,'./../concept-tracker') # okay for python terminal interactive console
sys.path.insert(1,'/Users/danielmlow/Dropbox (MIT)/datum/concept-tracker') # needed for jupyter interactive panel #TODO install
from concept_tracker.utils.datetime_conversions import list_months_between_interval
from sklearn.model_selection import train_test_split
pd.set_option("display.max_columns", None)


In [46]:
import datetime
ts = datetime.datetime.utcnow().strftime('%y-%m-%dT%H-%M-%S')

In [47]:

input_dir = './../../../data/ctl/input/'
output_dir = 'output/'


In [48]:
import datetime
def generate_timestamp(format = '%y-%m-%dT%H-%M-%S'):
	ts = datetime.datetime.utcnow().strftime(format) # so you don't overwrite, and save timestamp
	return ts

# Split into train-val-test sets

### From 2016 when we have tags (from 2016-03-05)

In [49]:
year_months = list_months_between_interval(start='2017-01-01', end='2023-01-01')
years = [2017, 2018, 2019, 2022] #removed 2020 and 2021, heavily biased by the pandemic

In [50]:
%%time
convo_ids_all = [] 
years_all = []
for year in years:
    metadata_i = pd.read_parquet(input_dir+f'raw_data/tags_risk_{year}_dummyvars.gzip', engine='pyarrow')        
    convo_ids = metadata_i['conversation_id'].values
    convo_ids_all.append(convo_ids)
    years_all.append([year]*len(convo_ids))

    

KeyboardInterrupt: 

In [ ]:
convo_ids_all = [n for i in convo_ids_all for n in i]
years_all = [n for i in years_all for n in i]


In [ ]:
len(convo_ids_all)

4477900

In [ ]:
# convo_train, years_train, convo_val, years_val, convo_test, years_test = train_valid_test_split(df, target = 'SalePrice', 
                                                                            # train_size=0.8, valid_size=0.1, test_size=0.1)


In [ ]:
convo_ids_train, convo_ids_valtest, years_train, years_valtest = train_test_split(convo_ids_all, years_all, test_size = 0.2, random_state=123)
convo_ids_val, convo_ids_test, years_val, years_test = train_test_split(convo_ids_valtest, years_val_test, test_size = 0.5, random_state=123)


In [ ]:
print('train:', len(convo_ids_train))
print('val:', len(convo_ids_val))
print('test:', len(convo_ids_test))

train: 3582320
val: 447790
test: 447790


In [ ]:
from collections import Counter
import matplotlib.pyplot as plt
for years_set, name in zip([years_train, years_val, years_test], ['train', 'val', 'test']):
    counts = Counter(years_set)
    print(name, counts)
    # plt.bar(x=counts.keys(), height = counts.values())
    # plt.title(name)
    # plt.show()

train Counter({2022: 993462, 2019: 954289, 2018: 906818, 2017: 727751})
val Counter({2022: 124347, 2019: 119737, 2018: 113223, 2017: 90483})
test Counter({2022: 124589, 2019: 119604, 2018: 113129, 2017: 90468})


### Obtain smaller set for concurrent validity

In [ ]:
convo_ids_train90, convo_ids_train10, years_train90, years_train10 = train_test_split(convo_ids_train, years_train, test_size = 0.1, random_state=123)


In [ ]:
print(len(convo_ids_train90), len(convo_ids_train10))

3224088 358232


# From train_10 create train10 train, val, test pkl splits

In [ ]:
convo_ids_train10_train, convo_ids_train10_valtest, years_train10_train, years_train10_valtest = train_test_split(convo_ids_train_10, years_train10, test_size = 0.2, random_state=123)
convo_ids_train10_val, convo_ids_train10_test, years_train10_val, years_train10_test = train_test_split(convo_ids_train10_valtest, years_train10_valtest, test_size = 0.5, random_state=123)


In [ ]:
print(len(convo_ids_train10_train), len(convo_ids_train10_val),len(convo_ids_train10_test))

286585 35823 35824


In [ ]:
sets = [
    [years_train, convo_ids_train], 
    [years_val, convo_ids_val], 
    [years_test, convo_ids_test], 
    [years_train90, convo_ids_train90], 
    [years_train10_train, convo_ids_train10_train],
    [years_train10_val, convo_ids_train10_val],
    [years_train10_test, convo_ids_train10_test],
    ]

In [ ]:
sets_names = [
    'train', 
    'val', 
    'test', 
    'train90', 
    'train10_train',
    'train10_val',
    'train10_test',
]


In [ ]:
input_dir

'./../../../data/ctl/input/raw_data/'

In [ ]:


os.makedirs('./data/input/train_test_split_ids/', exist_ok=True)

In [ ]:
d

{2019: 3831924, 2018: 1967346, 2022: 6907579, 2017: 1012814}

In [ ]:
years = ['a','b','a','b','a','b']
convo_ids = [1,2,3,4,5,6]

d = defaultdict(list)
for A, B in zip(years, convo_ids):
    d[A].append(B)
    
d

defaultdict(list, {'a': [1, 3, 5], 'b': [2, 4, 6]})

In [ ]:
%%time 
# 19 s
run_this = False

if run_this:

    for (years, convo_ids), name in zip(sets, sets_names):
        d = defaultdict(list)
        for A, B in zip(years, convo_ids):
            d[A].append(B)

        with open(input_dir+f'train_test_split_ids/{name}.pkl', 'wb') as f:
            pickle.dump(d, f)



CPU times: user 6 µs, sys: 2 µs, total: 8 µs
Wall time: 14.1 µs


# Build metadata.csv

### Choose tags

In [ ]:
high_risk_tags = ['ir_flag', 'active_rescue','suicidal_capability',
       'suicidal_desire', 'suicidal_intent', 'timeframe', 'perceived_risk']

In [ ]:
tags_alphabetical = ['3rd_party',
 'abuse_emotional',
 'abuse_physical',
 'abuse_sexual',
 # 'abuse_unspecified',
 'anxiety',
 'bereavement',
 'bully',
 'covid_19',
 'depressed',
 'domestic_violence',
 'eating',
 'election',
 'family',
 'financial',
 'friend',
 'gender',
 'homeless',
 'homicide',
 'immigration',
 'isolated',
 'medical',
 'medication',
 'mental',
 'military',
 'none',
 'other',
 'out_of_country',
 'prank',
 'psychiatric_hospitalization',
 'racism',
 'relationship',
 'school',
 'self_harm',
 'sexual_assault',
 'stress',
 'substance',
 'suicide',
 'testing',
 'therapist_psychiatrist',
 'work']

In [ ]:

cols_to_merge_d = {
     'work':'work_financial', #only exists pre 2018
     'financial':'work_financial', #only exists pre 2018
    'sexual_assault':'abuse_sexual', #changed names
    'anxiety':'anxiety_stress', #was an option for counselors "Anxiety/Stress"
    'stress':'anxiety_stress', 

     # 'abuse_phyiscal':'abuse_physical',
     # 'abuse_physical':'abuse_physical',
     # 'physical_abuse':'abuse_physical',
     # 'abuse_sexual':'abuse_sexual',
     # 'abuse_unspecified':'abuse',
     # 'domestic_violence':'domestic_violence',
     # 'sexual_abuse':'abuse_sexual',

     # 'alcohol': 'substance',
     # 'relationship_violence':'relationship',
     # 'family':'relationship',
     # 'friend':'relationship',
     'hunger':'eating_body_image',
    'eating':'eating_body_image',
    
     'gender':'gender_sexual_identity'
    }

def merge_cols(df, cols_to_merge_d):
    
    for key, value in cols_to_merge_d.items():
        if not value in df.columns:
            df[value] = 0
        try:
            df[value] += df[key]
        except:
            pass
    for key in cols_to_merge_d.keys():
        try: df = df.drop(key,axis=1)
        except:
            pass
    return df

In [ ]:
tags_high_freq = [
    'depressed',
 'relationship',
 'anxiety',
 'suicide',
 'none',
 'isolated',
 # 'other',
 'self_harm',
 'bereavement',
 # '3rd_party',
 'eating',
 'abuse_emotional',
 'family',
 # 'testing',
 'bully',
 'substance',
 'abuse_sexual',
 'covid_19',
 'gender',
 'abuse_physical',
 'stress',
 'friend',
 'school',
 # 'abuse_unspecified',
 # 'prank',
 # 'mental',
 'work',
 'financial',
 # 'medical',
 'therapist_psychiatrist',
 'medication',
 'sexual_assault',
 # 'homeless',
 # 'domestic_violence',#low frequency
 # 'psychiatric_hospitalization', #4500 in 2017
 # 'racism', #3k across the years
 # 'election', #2.5k across the years
 # 'military',
 # 'homicide',
 # 'immigration',
 # 'out_of_country'
]




# create a dataset for concurrent
concurrent_variables = np.unique(high_risk_tags+tags_high_freq+list(cols_to_merge_d.values()))
concurrent_variables        


array(['abuse_emotional', 'abuse_physical', 'abuse_sexual',
       'active_rescue', 'anxiety', 'anxiety_stress', 'bereavement',
       'bully', 'covid_19', 'depressed', 'eating', 'eating_body_image',
       'family', 'financial', 'friend', 'gender',
       'gender_sexual_identity', 'ir_flag', 'isolated', 'medication',
       'none', 'perceived_risk', 'relationship', 'school', 'self_harm',
       'sexual_assault', 'stress', 'substance', 'suicidal_capability',
       'suicidal_desire', 'suicidal_intent', 'suicide',
       'therapist_psychiatrist', 'timeframe', 'work', 'work_financial'],
      dtype='<U22')

In [ ]:
# 'train10_train' was done 

os.listdir(input_dir)


    

In [ ]:
%%time
# create DF from convo IDs

run_this = False


if run_this:

	for set_name in ['train10_train', 'train10_val', 'train10_test']:
	# for set_name in ['train10_train']:

		with open(input_dir+f'train_test_split_ids/{set_name}.pkl', 'rb') as f:
			d = pickle.load(f)

		df_set = []

		for year in [2017,2018,2019,2022]:
			d_year_ids = d.get(year)
			df_year = pd.read_parquet(input_dir+f'raw_data/tags_risk_{year}_dummyvars.gzip', engine='pyarrow')        
			df_year_set = df_year[df_year['conversation_id'].isin(d_year_ids)]
			df_set.append(df_year_set)

		df_set = pd.concat(df_set).reset_index(drop=True)    
		os.makedirs(input_dir+f'datasets', exist_ok=True)
		df_set.to_csv(input_dir+f'datasets/{set_name}_metadata.csv')

CPU times: user 24.3 s, sys: 5.8 s, total: 30.1 s
Wall time: 27.9 s


# metadata_csv to train10_train_concurrent_metadata (balanced 3k each crisis)


In [ ]:
# Load
df_set = pd.read_csv(input_dir+f'datasets/{set_name}_metadata.csv', index_col = 0)
df_set.shape

(35824, 61)

In [ ]:
df_set.columns

Index(['conversation_id', 'engaged', 'conversation_start_time_utc',
       'addedToQueue', 'takenFromQueue', 'areacode', 'type', 'carrier',
       'prank_ban', 'mandatory_report', 'risk', 'suicidality',
       'suicidality_sorted', 'ir_flag', 'active_rescue', 'active_rescue2',
       'value', 'tags', 'tags_sorted', '3rd_party', 'abuse_emotional',
       'abuse_physical', 'abuse_sexual', 'anxiety', 'bereavement', 'bully',
       'covid_19', 'depressed', 'domestic_violence', 'eating', 'election',
       'friend', 'gender', 'homeless', 'homicide', 'isolated', 'medical',
       'medication', 'military', 'none', 'other', 'prank', 'relationship',
       'self_harm', 'substance', 'suicide', 'testing', 'suicidal_capability',
       'suicidal_desire', 'suicidal_intent', 'timeframe', 'perceived_risk',
       'convo_year', 'convo_month', 'convo_dayofweek', 'convo_hour',
       'abuse_unspecified', 'racism',
       'suicidal_desiGreetings my name is Mark, and I'm here to help. It sounds like suici

In [ ]:
concurrent_variables_in_df = [n for n in variables_of_interest if n in df_set.columns]

counts = df_set[concurrent_variables_in_df].sum()
counts = pd.DataFrame(counts).reset_index()
counts.columns = ['construct', 'count']
counts.sort_values('count')[::-1]

,construct,count
8,depressed,7861.0
16,relationship,7013.0
4,anxiety,6301.0
22,suicide,5882.0
20,suicidal_desire,5023.0
15,none,4666.0
13,isolated,3901.0
17,self_harm,2462.0
21,suicidal_intent,2381.0
19,suicidal_capability,1728.0


In [ ]:
display((df_set[concurrent_variables_in_df].isna().sum()/df_set.shape[0]).round(2))

abuse_emotional        0.00
abuse_physical         0.00
abuse_sexual           0.00
active_rescue          0.00
anxiety                0.00
bereavement            0.00
bully                  0.00
covid_19               0.26
depressed              0.00
eating                 0.00
friend                 0.80
gender                 0.00
ir_flag                0.00
isolated               0.00
medication             0.80
none                   0.00
relationship           0.00
self_harm              0.00
substance              0.00
suicidal_capability    0.00
suicidal_desire        0.00
suicidal_intent        0.00
suicide                0.00
timeframe              0.00
dtype: float64

In [ ]:
df_set.shape

(35824, 61)

In [ ]:
df_set2 = merge_cols(df_set, cols_to_merge_d)

In [ ]:
set(df_set2.columns) - set(df_set.columns)

set()

In [ ]:
concurrent_variables_in_df = [n for n in concurrent_variables if n in df_set2.columns]


In [ ]:
(df_set2[concurrent_variables_in_df].isna().sum()/df_set2.shape[0]).round(2)

abuse_emotional           0.00
abuse_physical            0.00
abuse_sexual              0.00
active_rescue             0.00
anxiety_stress            0.00
bereavement               0.00
bully                     0.00
covid_19                  0.26
depressed                 0.00
eating_body_image         0.00
friend                    0.80
gender_sexual_identity    0.00
ir_flag                   0.00
isolated                  0.00
medication                0.80
none                      0.00
relationship              0.00
self_harm                 0.00
substance                 0.00
suicidal_capability       0.00
suicidal_desire           0.00
suicidal_intent           0.00
suicide                   0.00
timeframe                 0.00
work_financial            0.00
dtype: float64

In [ ]:
concurrent_variables_in_df

['abuse_emotional',
 'abuse_physical',
 'abuse_sexual',
 'active_rescue',
 'anxiety_stress',
 'bereavement',
 'bully',
 'covid_19',
 'depressed',
 'eating_body_image',
 'friend',
 'gender_sexual_identity',
 'ir_flag',
 'isolated',
 'medication',
 'none',
 'relationship',
 'self_harm',
 'substance',
 'suicidal_capability',
 'suicidal_desire',
 'suicidal_intent',
 'suicide',
 'timeframe',
 'work_financial']

In [ ]:
counts = df_set2[concurrent_variables_in_df].sum()
counts = pd.DataFrame(counts).reset_index()
counts.columns = ['construct', 'count']
counts.sort_values('count')[::-1]

,construct,count
8,depressed,7861.0
16,relationship,7013.0
4,anxiety_stress,6301.0
22,suicide,5882.0
20,suicidal_desire,5023.0
15,none,4666.0
13,isolated,3901.0
17,self_harm,2462.0
21,suicidal_intent,2381.0
19,suicidal_capability,1728.0


In [ ]:
counts.sort_values('count')[::-1]['construct'].values

array(['depressed', 'relationship', 'anxiety_stress', 'suicide',
       'suicidal_desire', 'none', 'isolated', 'self_harm',
       'suicidal_intent', 'suicidal_capability', 'bereavement',
       'eating_body_image', 'abuse_emotional', 'timeframe', 'bully',
       'substance', 'abuse_sexual', 'friend', 'abuse_physical',
       'gender_sexual_identity', 'ir_flag', 'active_rescue', 'medication',
       'covid_19', 'work_financial'], dtype=object)

In [ ]:
concurrent_analysis_vars = concurrent_variables_in_df.copy()
drop_cols = ['work_financial', 'covid_19', 'medication', 'active_rescue', 'ir_flag']
[concurrent_analysis_vars.remove(n) for n in drop_cols]
concurrent_analysis_vars

['abuse_emotional',
 'abuse_physical',
 'abuse_sexual',
 'anxiety_stress',
 'bereavement',
 'bully',
 'depressed',
 'eating_body_image',
 'friend',
 'gender_sexual_identity',
 'isolated',
 'none',
 'relationship',
 'self_harm',
 'substance',
 'suicidal_capability',
 'suicidal_desire',
 'suicidal_intent',
 'suicide',
 'timeframe']

In [ ]:
drop_cols = ['work_financial', 'covid_19', 'medication', 'active_rescue', 'ir_flag']
df_set = df_set.drop(drop_cols, axis=1)

In [ ]:
print(len(concurrent_analysis_vars))

20


In [ ]:
# Balance: Take n for each construct and concat. This is useful for one vs all classification. 
concurrent_df_n = 3000

concurrent_df = []
for variable in concurrent_analysis_vars:
    df_set2_variable_i = df_set2[df_set2[variable]==1].sample(n = concurrent_df_n)
    concurrent_df.append(df_set2_variable_i)
    
concurrent_df = pd.concat(concurrent_df).reset_index(drop=True)

In [ ]:
counts = concurrent_df[concurrent_variables_in_df].sum()
counts = pd.DataFrame(counts).reset_index()
counts.columns = ['construct', 'count']
counts.sort_values('count')[::-1]

,construct,count
22,suicide,26725.0
20,suicidal_desire,24551.0
8,depressed,23183.0
16,relationship,20677.0
21,suicidal_intent,16324.0
4,anxiety_stress,16238.0
13,isolated,14474.0
19,suicidal_capability,13495.0
17,self_harm,10138.0
23,timeframe,7195.0


In [ ]:
concurrent_df = concurrent_df.drop("suicidal_desiGreetings my name is Mark, and I'm here to help. It sounds like suicidal_desire", axis=1)




### Save DF 

In [ ]:
input_dir

'./../../../data/ctl/input/'

In [ ]:
concurrent_df = concurrent_df.drop_duplicates('conversation_id')
concurrent_df.to_csv(input_dir+'../datasets/train10_train_concurrent_metadata.csv')

In [ ]:
# Load
df_set = pd.read_csv(input_dir+f'datasets/{set_name}_metadata.csv', index_col = 0)
df_set.shape

# risk assessment small

- train: 6k, 2k of each level of risk
- val: 3% of train10_val
- test: 3% of train10_test


In [76]:
def get_true_risk_a(row):
	if (row['ir_flag'] > 0 or row['timeframe'] > 0 or row['active_rescue'] > 0):
		return 3 # high risk
	
	elif ('suicidal_desire' in row['suicidality'] or 'suicidal_intent' in row['suicidality'] or 'suicidal_capability' in row['suicidality']  or row['suicide']>0 or row['self_harm']>0) \
			and (row['3rd_party'] !=1 and row['testing'] != 1 and row['prank'] != 1): 
		return 2 # medium risk
	else:
		return 1 # normal risk


def get_true_risk_b(row):
	if (row['active_rescue'] > 0):
		return 4 # active rescue
	
	elif (row['ir_flag'] > 0 or row['timeframe'] > 0):
		return 3 # high risk
	
	elif ('suicidal_desire' in row['suicidality'] or 'suicidal_intent' in row['suicidality'] or 'suicidal_capability' in row['suicidality']  or row['suicide']>0 or row['self_harm']>0) \
			and (row['3rd_party'] !=1 and row['testing'] != 1 and row['prank'] != 1): 
		return 2 # medium risk
	else:
		return 1 # normal risk
	

def get_true_risk_c(row):
	if (row['active_rescue'] > 0):
		return 6 # active rescue
	
	elif (row['ir_flag'] > 0):
		return 5 # high risk
	
	elif (row['timeframe'] > 0):
		return 4 # high risk
	
	elif (row['suicidal_intent']>0 or row['suicidal_capability']>0) and (row['3rd_party'] !=1 and row['testing'] != 1 and row['prank'] != 1):
		return 3
	
	elif (row['suicidal_desire']>0 or row['self_harm']>0 or row['suicide']>0) and (row['3rd_party'] !=1 and row['testing'] != 1 and row['prank'] != 1):
		return 2
	else: 
		return 1
	# elif (row['suicide'] == 0) and (row['3rd_party'] !=1 and row['testing'] != 1 and row['prank'] != 1):
	# 	# and all the above will == 0. 
		return 1 # normal risk


In [66]:
# OUt of active rescues how many had timeframe tagged
metadata_df_i = pd.read_csv(input_dir+f'datasets/train10_test_metadata.csv', index_col=0)

display(metadata_df_i[metadata_df_i['active_rescue']==1]['timeframe'].value_counts())
display(metadata_df_i[metadata_df_i['active_rescue']==1]['ir_flag'].value_counts())
display(metadata_df_i[metadata_df_i['ir_flag']==1]['timeframe'].value_counts())
display(metadata_df_i[metadata_df_i['suicidal_intent']==1]['suicidal_desire'].value_counts())
display(metadata_df_i[metadata_df_i['suicide']==1]['suicidal_intent'].value_counts())
display(metadata_df_i[metadata_df_i['suicidal_intent']==1]['suicidal_capability'].value_counts())

timeframe
1    196
0     22
Name: count, dtype: int64

ir_flag
0    207
1     11
Name: count, dtype: int64

timeframe
1    276
0     74
Name: count, dtype: int64

suicidal_desire
1    2381
0      44
Name: count, dtype: int64

suicidal_intent
0    3389
1    2422
Name: count, dtype: int64

suicidal_capability
1    1684
0     741
Name: count, dtype: int64

In [100]:
tags_of_interest = ['abuse_emotional', 'abuse_physical', 'abuse_sexual',
        'anxiety', 'anxiety_stress', 'bereavement',
       'bully', 'covid_19', 'depressed', 'eating', 'eating_body_image',
       'family', 'financial', 'friend', 'gender',
       'gender_sexual_identity', 'isolated', 'medication',
       'none', 'relationship', 'school',  'sexual_assault',
       'stress', 'substance', 
         'therapist_psychiatrist',
        'work', 'work_financial', 
		'perceived_risk','self_harm','suicide','suicidal_desire', 'suicidal_intent','suicidal_capability','timeframe', 'ir_flag', 'active_rescue',
		'suicide_ladder_a', 'suicide_ladder_b', 'suicide_ladder_c']




tags_of_interest_in_df = [n for n in tags_of_interest if n in metadata_df_i.columns]

metadata_df_i[tags_of_interest_in_df].sum()

abuse_emotional          1672.0
abuse_physical            905.0
abuse_sexual             1242.0
anxiety                 15083.0
bereavement              2135.0
bully                    1460.0
covid_19                   85.0
depressed               18649.0
eating                   1631.0
friend                   1286.0
gender                    976.0
isolated                 9386.0
medication                218.0
none                    11087.0
relationship            16593.0
substance                1319.0
perceived_risk         192517.0
self_harm                5969.0
suicide                 13638.0
suicidal_desire         11600.0
suicidal_intent          5522.0
suicidal_capability      4066.0
timeframe                1657.0
ir_flag                   800.0
active_rescue             541.0
suicide_ladder_a       105315.0
suicide_ladder_b       105856.0
suicide_ladder_c       113077.0
dtype: float64

In [81]:
metadata_df_i = pd.read_csv(input_dir+f'datasets/train10_test_metadata.csv', index_col=0)

metadata_df_i = metadata_df_i.drop("suicidal_desiGreetings my name is Mark, and I'm here to help. It sounds like suicidal_desire", axis=1)


metadata_df_i['suicide_ladder_a'] = [get_true_risk_a(row) for index, row in metadata_df_i.iterrows()]
metadata_df_i['suicide_ladder_b'] = [get_true_risk_b(row) for index, row in metadata_df_i.iterrows()]
metadata_df_i['suicide_ladder_c'] = [get_true_risk_c(row) for index, row in metadata_df_i.iterrows()]

In [82]:
display(metadata_df_i['suicide_ladder_a'].value_counts())
display(metadata_df_i['suicide_ladder_b'].value_counts())
display(metadata_df_i['suicide_ladder_c'].value_counts())


suicide_ladder_a
1    28409
2     6624
3      791
Name: count, dtype: int64

suicide_ladder_b
1    28409
2     6624
3      581
4      210
Name: count, dtype: int64

suicide_ladder_c
1    28409
2     4869
3     1755
5      322
4      259
6      210
Name: count, dtype: int64

In [97]:
# How many suicide risk do we have?

input_dir = './../../../data/ctl/input/'


for set_name in ['train10_train', 'train10_val', 'train10_test']:
	print(input_dir+f'datasets/{set_name}_50perc_metadata.csv')
	metadata_df_i = pd.read_csv(input_dir+f'datasets/{set_name}_metadata.csv', index_col=0)
	
	metadata_df_i = metadata_df_i.drop("suicidal_desiGreetings my name is Mark, and I'm here to help. It sounds like suicidal_desire", axis=1)
	
	
	metadata_df_i['suicide_ladder_a'] = [get_true_risk_a(row) for index, row in metadata_df_i.iterrows()]
	metadata_df_i['suicide_ladder_b'] = [get_true_risk_b(row) for index, row in metadata_df_i.iterrows()]
	metadata_df_i['suicide_ladder_c'] = [get_true_risk_c(row) for index, row in metadata_df_i.iterrows()]

	if set_name == 'train10_train':
		metadata_df_i = metadata_df_i.sample(frac= 0.3, random_state = 123)
		metadata_df_i.to_csv(input_dir+f'datasets/{set_name}_30perc_metadata.csv')
		
	else:
		metadata_df_i = metadata_df_i.sample(frac= 0.15, random_state = 123)
		metadata_df_i.to_csv(input_dir+f'datasets/{set_name}_15perc_metadata.csv')


./../../../data/ctl/input/datasets/train10_train_50perc_metadata.csv
./../../../data/ctl/input/datasets/train10_val_50perc_metadata.csv
./../../../data/ctl/input/datasets/train10_test_50perc_metadata.csv


In [96]:
set_name

'train10_train_30perc'

In [89]:
display(metadata_df_i['suicide_ladder_a'].value_counts())
display(metadata_df_i['suicide_ladder_b'].value_counts())
display(metadata_df_i['suicide_ladder_c'].value_counts())


suicide_ladder_a
1    4285
2     961
3     128
Name: count, dtype: int64

suicide_ladder_b
1    4285
2     961
3      88
4      40
Name: count, dtype: int64

suicide_ladder_c
1    4285
2     691
3     270
5      51
6      40
4      37
Name: count, dtype: int64

# Find messages from metadata.csv files

In [90]:
import pandas as pd
import pickle


set_names = ['train10_train_30perc' ,'train10_val_15perc','train10_test_15perc']

In [101]:
# How many suicide risk do we have?


for set_name in set_names:
	metadata_df_i = pd.read_csv(input_dir+f'datasets/{set_name}_metadata.csv', index_col = 0)
	tags_of_interest_in_df = [n for n in tags_of_interest if n in metadata_df_i.columns]

	counts = metadata_df_i[tags_of_interest_in_df].sum()
	counts = pd.DataFrame(counts).reset_index()
	counts.columns = ['construct', 'count']
	display(counts.sort_values('count')[::-1])

	



,construct,count
16,perceived_risk,192517.0
27,suicide_ladder_c,113077.0
26,suicide_ladder_b,105856.0
25,suicide_ladder_a,105315.0
7,depressed,18649.0
14,relationship,16593.0
3,anxiety,15083.0
18,suicide,13638.0
19,suicidal_desire,11600.0
13,none,11087.0


,construct,count
16,perceived_risk,12039.0
27,suicide_ladder_c,7081.0
26,suicide_ladder_b,6611.0
25,suicide_ladder_a,6585.0
7,depressed,1146.0
14,relationship,1076.0
3,anxiety,960.0
18,suicide,878.0
19,suicidal_desire,747.0
13,none,722.0


,construct,count
16,perceived_risk,12047.0
27,suicide_ladder_c,7120.0
26,suicide_ladder_b,6631.0
25,suicide_ladder_a,6591.0
7,depressed,1218.0
14,relationship,1038.0
3,anxiety,951.0
18,suicide,890.0
19,suicidal_desire,761.0
13,none,703.0


In [102]:
display(metadata_df_i['tags_sorted'])
display(metadata_df_i[['tags_sorted', 'suicidality_sorted', 'perceived_risk', 'suicide', 'ir_flag', 'active_rescue']])
display(metadata_df_i[['ir_flag', 'active_rescue','active_rescue2' ]].sum())
display(metadata_df_i[['perceived_risk' ]].value_counts())

16654                                          ['anxiety']
4772                                              ['none']
25681                              ['anxiety' 'self_harm']
19893                      ['anxiety' 'depressed' 'other']
25518    ['bereavement' 'depressed' 'isolated' 'relatio...
                               ...                        
18003                                                   []
32980                                     ['bully' 'none']
15294                                                   []
30251                  ['anxiety' 'depressed' 'self_harm']
10444                                             ['none']
Name: tags_sorted, Length: 5374, dtype: object

,tags_sorted,suicidality_sorted,perceived_risk,suicide,ir_flag,active_rescue
16654,['anxiety'],[],2,0,0,0
4772,['none'],[],2,0,0,0
25681,['anxiety' 'self_harm'],[],3,0,0,0
19893,['anxiety' 'depressed' 'other'],[],2,0,0,0
25518,['bereavement' 'depressed' 'isolated' 'relatio...,[],2,0,0,0
...,...,...,...,...,...,...
18003,[],[],2,0,0,0
32980,['bully' 'none'],[],2,0,0,0
15294,[],[''],2,0,0,0
30251,['anxiety' 'depressed' 'self_harm'],[],3,0,0,0


ir_flag           53
active_rescue     40
active_rescue2     0
dtype: int64

perceived_risk
2                 4337
3                  775
4                  262
Name: count, dtype: int64

In [ ]:
# # Load
# df_set = pd.read_csv(input_dir+f'datasets/{set_name}_metadata.csv', index_col = 0)
# df_set.shape

# # concurrent_df  = pd.read_csv(input_dir+'datasets/train10_train_concurrent_metadata.csv', index_col = 0)



In [110]:
conversation_cols = [
    'conversation_id',
    'conversation_start_time_utc',
 'conversation_end_time_utc',
 'post_conversation_survey_end_time',
 'priority',
 'priority_predicted',
 'presenting_concern_message_id',
 'is_risk_assessed',
 'keyword',
 'num_wait_time_notifications',
 'language']

conversation_cols_to_drop = [e for e in conversation_cols if e not in ['conversation_id',
 'conversation_start_time_utc']]

In [ ]:
# metadata_df_ids = metadata_df_i['conversation_id'].values
# metadata_df_ids

array([2792548, 1304192, 3937242, ..., 2808053, 6960829, 7989846])

In [105]:
set_name = 'train10_train'

with open(input_dir+f'train_test_split_ids/{set_name}.pkl', 'rb') as f:
    d = pickle.load(f)

    

In [113]:



%%time
# 8min
# create training DF from convo IDs

run_this = True
if run_this:

	for set_name in set_names: 
		metadata_df = pd.read_csv(input_dir+f'datasets/{set_name}_metadata.csv', index_col = 0)

		metadata_df_ids = metadata_df['conversation_id'].values



		df_set = []
		df_unique_per_conversation_all = []
		years = [2017,2018,2019,2022]
		# if toy: 
		#     years = years[:3]

		for year in years:
			print(year, '=======')
			d_year_ids = d.get(year) # get list of conversation_ids in this year
			
			
			
			df_year = []
			for month in range(1,13):
				print(month)
				month = str(month).zfill(2)
				df_year_month_i = pd.read_parquet(input_dir+f'raw_data/messages_{year}-{month}.gzip', engine='pyarrow')       
				df_year.append(df_year_month_i)
			df_year = pd.concat(df_year).reset_index(drop=True) # all messages for that year
		
			# else:
			#     df_year = pd.read_parquet(input_dir+f'raw_data/messages_{year}.gzip', engine='pyarrow') 
				
			# df_set.to_parquet(input_dir+f'datasets/train10_train_concurrent_messages.gzip', compression = 'gzip', engine='pyarrow')        
			
			# choose conversation that are in metadata DF of interest
			
			df_year_set = df_year[df_year['conversation_id'].isin(metadata_df_ids)] # NEW
			# df_year_set = df_year[df_year['conversation_id'].isin(d_year_ids)] # OLD
			
			# these messages_df cols are unique within a conversation, so add to metadata_df and remove from messages_df to save space
			df_year_unique_per_conversation = df_year_set[conversation_cols] 
			df_year_unique_per_conversation = df_year_unique_per_conversation.drop_duplicates()
			
			
			# normalize formatting
			# concurrent_df = concurrent_df.astype(str).reset_index(drop=True)
			# df_year_unique_per_conversation = df_year_unique_per_conversation.astype(str).reset_index(drop=True)
			df_unique_per_conversation_all.append(df_year_unique_per_conversation)
			# df_year_unique_per_conversation['conversation_start_time_utc'] = pd.to_datetime(df_year_unique_per_conversation['conversation_start_time_utc'])
			# df_year_unique_per_conversation['conversation_end_time_utc'] = pd.to_datetime(df_year_unique_per_conversation['conversation_end_time_utc'])
			# df_year_unique_per_conversation['post_conversation_survey_end_time'] = pd.to_datetime(df_year_unique_per_conversation['post_conversation_survey_end_time'])
			
			
			
			# # todo merge with concurrent
			# if year == years[0]:            
			#     # concurrent_df['conversation_start_time_utc'] = pd.to_datetime(concurrent_df['conversation_start_time_utc'])
			#     concurrent_df = concurrent_df.merge(df_year_unique_per_conversation, on=['conversation_id', 'conversation_start_time_utc'] , how = 'left')    
			# else:           
			#     concurrent_df = concurrent_df.merge(df_year_unique_per_conversation, on=conversation_cols, how = 'left')
			
			
			df_year_set = df_year_set.drop(conversation_cols_to_drop, axis=1)
			
			# TODO: uncomment line below
			# df_year_set.to_parquet(input_dir+f'datasets/train10_train_concurrent_messages_{year}.gzip', compression = 'gzip', engine='pyarrow')        
			
			df_set.append(df_year_set)

		# concurrent_df.to_csv(input_dir+'datasets/train10_train_concurrent_metadata2.csv')
		df_set = pd.concat(df_set).reset_index(drop=True)        
		df_set.to_parquet(input_dir+f'datasets/{set_name}_messages.gzip', compression = 'gzip', engine='pyarrow')        
		df_year_texter = df_set[df_set['interaction']=='texter']
		df_year_texter.to_parquet(input_dir+f'datasets/{set_name}_messages_texter.gzip', compression = 'gzip', engine='pyarrow')        
		df_year_counselor = df_set[df_set['interaction']=='counselor']
		df_year_counselor.to_parquet(input_dir+f'datasets/{set_name}_messages_counselor.gzip', compression = 'gzip', engine='pyarrow')        
		
		df_unique_per_conversation_all = pd.concat(df_unique_per_conversation_all).reset_index(drop=True)        
		df_unique_per_conversation_all.to_parquet(input_dir+f'datasets/{set_name}_messages_{year}_columns_unique_to_convo.gzip', compression = 'gzip', engine='pyarrow')        
		
		
		

2017 =======
1
2
3
4
5
6
7
8
9
10
11
12
2018 =======
1
2
3
4
5
6
7
8
9
10
11
12


<timed exec>:32: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.


2019 =======
1
2
3
4
5
6
7
8
9
10
11
12
2022 =======
1
2
3
4
5
6
7
8
9
10
11
12


<timed exec>:82: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.


2017 =======
1
2
3
4
5
6
7
8
9
10
11
12
2018 =======
1
2
3
4
5
6
7
8
9
10
11
12


...

<timed exec>:82: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.


2017 =======
1
2
3
4
5
6
7
8
9
10
11
12
2018 =======
1
2
3
4
5
6
7
8
9
10
11
12


<timed exec>:32: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.


2019 =======
1
2
3
4
5
6
7
8
9
10
11
12
2022 =======
1
2
3
4
5
6
7
8
9
10
11
12
CPU times: user 10min 19s, sys: 11min 49s, total: 22min 9s
Wall time: 45min 6s


<timed exec>:82: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.


In [ ]:
# for set_name in set_names:
# 	df_set = pd.read_parquet(input_dir+f'datasets/{set_name}_messages.gzip', engine='pyarrow')        
# 	break





In [ ]:
# df_set

,conversation_id,conversation_start_time_utc,supervisor_notes,id,actor_id,interaction,message_id,message_timestamp_utc,message
0,700061,2017-01-01 00:12:12,None,1549031,331196,texter,28225720,2017-01-01 00:12:12,Go
1,700061,2017-01-01 00:12:12,None,1549031,331196,texter,28225758,2017-01-01 00:12:54,I feel swamped with marriage like I can't hand...
2,700061,2017-01-01 00:12:12,None,1549036,276284,counselor,28225806,2017-01-01 00:14:06,"Hi, i'm [scrubbed]. You sound upset. Can yo..."
3,700061,2017-01-01 00:12:12,None,1549031,331196,texter,28225940,2017-01-01 00:16:52,"It started this early afternoon, with my husba..."
4,700061,2017-01-01 00:12:12,None,1549036,276284,counselor,28225990,2017-01-01 00:17:54,So you and your husband had an argument that s...
...,...,...,...,...,...,...,...,...,...
755817,8031100,2022-12-31 23:08:23,"aes256cbc#v0#{""iv"":""nGfZ2tgFfjdNkK9uYAGTtg=="",...",15407130,2029540,texter,258891656,2022-12-31 23:48:19,thank you o think that's all
755818,8031100,2022-12-31 23:08:23,"aes256cbc#v0#{""iv"":""nGfZ2tgFfjdNkK9uYAGTtg=="",...",15407216,3477498,counselor,258891719,2022-12-31 23:48:57,You are so welcome! Are you ready to take on t...
755819,8031100,2022-12-31 23:08:23,"aes256cbc#v0#{""iv"":""nGfZ2tgFfjdNkK9uYAGTtg=="",...",15407216,3477498,counselor,258892087,2022-12-31 23:53:49,Just wanted to check in and see if now is a go...
755820,8031100,2022-12-31 23:08:23,"aes256cbc#v0#{""iv"":""nGfZ2tgFfjdNkK9uYAGTtg=="",...",15407130,2029540,texter,258892141,2022-12-31 23:54:24,i'm done i think thank you for your time and s...


In [ ]:
cols_not_added = [n for n in df_year_month_i.columns if n not in df_set.columns]
cols_not_added


['conversation_end_time_utc',
 'post_conversation_survey_end_time',
 'priority',
 'priority_predicted',
 'presenting_concern_message_id',
 'is_risk_assessed',
 'keyword',
 'num_wait_time_notifications',
 'language']

In [ ]:
for col in df_year_month_i[cols_not_added].columns:
	print(col)
	display(df_year_month_i[cols_not_added][col].value_counts(normalize=True).iloc[:10])


conversation_end_time_utc


2022-12-27 07:05:03    0.000255
2022-12-30 06:00:03    0.000253
2022-12-21 07:00:04    0.000250
2022-12-02 05:45:03    0.000246
2022-12-09 18:15:03    0.000239
2022-12-19 03:45:03    0.000237
2022-12-17 06:05:03    0.000222
2022-12-24 05:00:03    0.000222
2022-12-26 22:25:03    0.000218
2022-12-05 22:50:03    0.000215
Name: conversation_end_time_utc, dtype: float64

post_conversation_survey_end_time


2022-12-23 05:05:03    0.000277
2022-12-10 03:50:03    0.000267
2022-12-26 22:25:03    0.000260
2022-12-02 07:35:02    0.000256
2022-12-28 03:40:03    0.000256
2022-12-31 08:00:03    0.000248
2022-12-23 00:00:03    0.000245
2022-12-06 04:25:03    0.000240
2022-12-11 07:25:02    0.000237
2022-12-30 07:00:03    0.000233
Name: post_conversation_survey_end_time, dtype: float64

priority


...

HOME      0.276424
Home      0.105379
Hello     0.087118
Hi        0.066632
I         0.037215
Help      0.028122
Hey       0.024713
TikTok    0.022631
hello     0.020220
hi        0.019060
Name: keyword, dtype: float64

num_wait_time_notifications


0     0.709826
1     0.139775
2     0.060865
3     0.040903
4     0.026933
5     0.021069
6     0.000361
14    0.000095
15    0.000085
10    0.000045
Name: num_wait_time_notifications, dtype: float64

language


en    0.992721
es    0.007279
Name: language, dtype: float64

In [ ]:
metadata_df

,Unnamed: 0.1,conversation_id,engaged,conversation_start_time_utc,addedToQueue,takenFromQueue,areacode,type,carrier,prank_ban,mandatory_report,risk,suicidality,suicidality_sorted,ir_flag,active_rescue,active_rescue2,value,tags,tags_sorted,3rd_party,abuse_emotional,abuse_physical,abuse_sexual,anxiety,bereavement,bully,covid_19,depressed,domestic_violence,eating,election,friend,gender,homeless,homicide,isolated,medical,medication,military,none,other,prank,relationship,self_harm,substance,suicide,testing,suicidal_capability,suicidal_desire,suicidal_intent,timeframe,perceived_risk,convo_year,convo_month,convo_dayofweek,convo_hour,abuse_unspecified,racism,immigration,out_of_country
153377,153377,3109326,1,2019-04-20 16:21:39,2019-04-20 16:22:10,2019-04-20 16:22:45,NaN,Facebook,NaN,0,0,a:0:{},[],[],0,0,0,"a:3:{i:0;s:14:""abuse_physical"";i:1;s:15:""abuse...",['abuse_physical' 'abuse_emotional' 'relations...,['abuse_emotional' 'abuse_physical' 'relations...,0,1,1,0,0,0,0,0.0,0,NaN,0,NaN,NaN,0,NaN,NaN,0,NaN,NaN,NaN,0,0,0,1,0,0,0,0,0,0,0,0,2,2019,4,5,16,0.0,0.0,NaN,NaN
84648,84648,2032243,1,2018-06-06 23:34:50,2018-06-06 23:35:35,2018-06-06 23:36:32,201.0,SMS,AT&T Wireless,0,0,NaN,[''],[''],0,0,0,"a:1:{i:0;s:7:""testing"";}",['testing'],['testing'],0,0,0,0,0,0,0,NaN,0,NaN,0,NaN,NaN,0,NaN,NaN,0,NaN,NaN,NaN,0,0,0,0,0,0,0,1,0,0,0,0,2,2018,6,2,23,0.0,NaN,NaN,NaN
259197,259197,7564660,1,2022-08-19 04:10:00,2022-08-19 04:10:24,2022-08-19 04:15:07,240.0,SMS,"T-Mobile USA, Inc.",0,0,a:0:{},[],[],0,0,0,"a:1:{i:0;s:8:""isolated"";}",['isolated'],['isolated'],0,0,0,0,0,0,0,0.0,0,NaN,0,0.0,NaN,0,NaN,NaN,1,NaN,NaN,NaN,0,0,0,0,0,0,0,0,0,0,0,0,2,2022,8,4,4,0.0,0.0,0.0,0.0
59501,59501,1638136,1,2018-01-07 02:35:40,2018-01-07 02:36:05,2018-01-07 02:42:32,301.0,SMS,T-Mobile,0,0,a:0:{},[],[],0,0,0,"a:6:{i:0;s:5:""bully"";i:1;s:7:""anxiety"";i:2;s:9...",['bully' 'anxiety' 'depressed' 'gender' 'isola...,['anxiety' 'bully' 'depressed' 'gender' 'isola...,0,0,0,0,1,0,1,NaN,1,NaN,0,NaN,NaN,1,NaN,NaN,1,NaN,NaN,NaN,0,0,0,1,0,0,0,0,0,0,0,0,2,2018,1,6,2,0.0,NaN,NaN,NaN
2696,2696,741645,0,2017-01-28 21:27:32,NaN,2017-01-28 21:28:05,360.0,MobileMessenger,Verizon Wireless,0,0,NaN,[''],[''],0,0,0,NaN,[],[],0,0,0,0,0,0,0,0.0,0,0.0,0,0.0,0.0,0,0.0,0.0,0,0.0,0.0,0.0,0,0,0,0,0,0,0,0,0,0,0,0,2,2017,1,5,21,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
153918,153918,3118020,1,2019-04-23 09:53:52,2019-04-23 09:54:23,2019-04-23 10:02:29,856.0,SMS,"T-Mobile USA, Inc.",0,0,N;,[],[],0,0,0,"a:2:{i:0;s:12:""relationship"";i:1;s:15:""abuse_e...",['relationship' 'abuse_emotional'],['abuse_emotional' 'relationship'],0,1,0,0,0,0,0,0.0,0,NaN,0,NaN,NaN,0,NaN,NaN,0,NaN,NaN,NaN,0,0,0,1,0,0,0,0,0,0,0,0,2,2019,4,1,9,0.0,0.0,NaN,NaN
165335,165335,3301248,1,2019-06-15 04:14:58,2019-06-15 04:15:54,2019-06-15 04:16:00,513.0,SMS,AT&T Wireless,0,0,a:0:{},[],[],0,0,0,"a:1:{i:0;s:7:""anxiety"";}",['anxiety'],['anxiety'],0,0,0,0,1,0,0,0.0,0,NaN,0,NaN,NaN,0,NaN,NaN,0,NaN,NaN,NaN,0,0,0,0,0,0,0,0,0,0,0,0,2,2019,6,5,4,0.0,0.0,NaN,NaN
209152,209152,6735998,1,2022-01-08 11:22:02,2022-01-08 11:24:26,2022-01-08 12:15:31,845.0,SMS,"T-Mobile USA, Inc.",0,0,a:0:{},[],[],0,0,0,"a:1:{i:0;s:7:""anxiety"";}",['anxiety'],['anxiety'],0,0,0,0,1,0,0,0.0,0,NaN,0,0.0,NaN,0,NaN,NaN,0,NaN,NaN,NaN,0,0,0,0,0,0,0,0,0,0,0,0,2,2022,1,5,11,0.0,0.0,0.0,0.0
56142,56142,1585514,1,2017-12-19 04:03:35,2017-12-19 04:05:45,2017-12-19 04:27:52,312.0,SMS,"T-Mobile USA, Inc.",0,0,NaN,[''],[''],0,0,0,"a:1:{i:0;s:5:""other"";}",['other'],['other'],0,0,0,0,0,0,0,0.0,0,0.0,0,0.0,0.0,0,0.0,0.0,0,0.0,0.0,0.0,0,1,0,0,0,0,0,0,0,0,0,0,2,2017,12,1,4,NaN,NaN,NaN,NaN


# Build tokenized DF

In [ ]:
import numpy as np

In [ ]:
# TODO: merge active_si and passive_si, change name of self-injury


constructs_to_measure = ['self-injury',
 'active_si',
 'passive_si',
 'bully',
 'abuse_physical',
 'abuse_sexual',
 'relationships',
 'grief_bereavement',
 'loneliness_isolated',
 'anxiety',
 'depressed_mood',
 'gender_sexual_identity',
 'eating_disorder',
 'substance_use']



# values = ['']*len(constructs_to_measure)
# ctl_tags = dict(zip(constructs_to_measure, values))
ctl_tags_d = {'self-injury': 'self_harm',
 'active_si': 'suicide',
 'passive_si': 'suicide',
 'bully': 'bully',
 'abuse_physical': 'abuse_physical',
 'abuse_sexual': 'abuse_sexual',
 'relationships': 'relationship',
 'grief_bereavement': 'bereavement',
 'loneliness_isolated': 'isolated',
 'anxiety': 'anxiety_stress',
 'depressed_mood': 'depressed',
 'gender_sexual_identity': 'gender_sexual_identity',
 'eating_disorder': 'eating_body_image',
 'substance_use': 'substance'}

ctl_tags = np.unique(list(ctl_tags_d.values())).tolist()
ctl_tags

['abuse_physical',
 'abuse_sexual',
 'anxiety_stress',
 'bereavement',
 'bully',
 'depressed',
 'eating_body_image',
 'gender_sexual_identity',
 'isolated',
 'relationship',
 'self_harm',
 'substance',
 'suicide']

In [ ]:
run_this = False
if run_this:

    df_messages = pd.read_parquet(input_dir+f'datasets/train10_train_concurrent_messages_texter.gzip', engine='pyarrow')        
    df_metadata = pd.read_csv(input_dir+f'datasets/train10_train_concurrent_metadata.csv', index_col = 0)
    
    
    df_metadata = df_metadata.drop_duplicates('conversation_id').reset_index(drop=True)
    print(df_metadata.shape, df_messages.shape)

    # Create a subset of metadata that has at least n_samples of a given construct
    n_samples = 100

    df_metadata_subsample = []

    for n in ctl_tags:
        df_metadata_tag = df_metadata[df_metadata[n]==1].sample(n=n_samples,random_state=123)
        df_metadata_subsample.append(df_metadata_tag)

    df_metadata_subsample = pd.concat(df_metadata_subsample).reset_index(drop=True)
    print(df_metadata_subsample.shape)
    df_metadata_subsample_tags = df_metadata_subsample[['conversation_id']+ctl_tags]
    df_metadata_subsample_tags = df_metadata_subsample_tags.drop_duplicates().reset_index(drop=True)
    # print(df_metadata_subsample_tags.shape)
    # df_metadata_subsample_tags.to_csv(input_dir+"multioutput_"+training_lexicon+'.csv')
    
    # Find messages for the subsample
    df_messages_subsample = df_messages[df_messages['conversation_id'].isin(df_metadata_subsample_tags['conversation_id'])]
    df_messages_subsample = df_messages_subsample.drop_duplicates('message_id').reset_index(drop=True)
    print(df_metadata_subsample_tags.shape)

    # df_test_multioutput_subsample = df_messages_subsample.merge(df_metadata_subsample_tags,on='conversation_id', how='left')
    # print(df_test_multioutput_subsample.shape)
    # df_test_multioutput_subsample['text'] = df_test_multioutput_subsample['message'] 

    # # add label column for multi-output training
    # # df_test_multioutput_subsample = pd.read_csv(input_dir+"multioutput_ctl_train10_1266_"+training_lexicon+'.csv', index_col = 0)
    # df_test_multioutput_subsample['labels'] = [np.array(n) for n in df_test_multioutput_subsample[ctl_tags].values]
    # df_test_multioutput_subsample = df_test_multioutput_subsample.drop_duplicates('message_id')
    # df_test_multioutput_subsample.to_csv(input_dir+"multioutput_ctl_train10_1266_"+training_lexicon+'.csv')


(49096, 61) (4100865, 9)
(1300, 61)
(1287, 14)


In [ ]:
'SURVEY' in top_texter_messages

False

In [ ]:
counts = pd.DataFrame(df_messages_subsample['message'].value_counts())

# after reviewing, remove top 100 messages, which are a lot of keywords to log in. 
top_texter_messages = counts.iloc[:100].index.tolist()
top_texter_messages.remove('I want to die')
top_texter_messages.remove('I need help')
print(df_messages_subsample.shape)
df_messages_subsample = df_messages_subsample[~df_messages_subsample['message'].isin(top_texter_messages)]
print(df_messages_subsample.shape)

(35812, 9)
(31130, 9)


In [ ]:
df_messages_subsample

,conversation_id,conversation_start_time_utc,supervisor_notes,id,actor_id,interaction,message_id,message_timestamp_utc,message
0,705014,2017-01-04 06:38:47,None,1559869,312760,texter,28421990,2017-01-04 06:38:47,SURVEY
1,705014,2017-01-04 06:38:47,None,1559869,312760,texter,28422026,2017-01-04 06:39:13,I want to continue the talk I was having before
2,705014,2017-01-04 06:38:47,None,1559869,312760,texter,28422357,2017-01-04 06:43:50,I was literally peed on during school and othe...
3,705014,2017-01-04 06:38:47,None,1559869,312760,texter,28422932,2017-01-04 06:52:05,I need somebody to talk to
4,705014,2017-01-04 06:38:47,None,1559869,312760,texter,28423083,2017-01-04 06:54:13,I'm thinking that at my school I'm just going ...
...,...,...,...,...,...,...,...,...,...
35807,8026068,2022-12-30 11:16:14,"aes256cbc#v0#{""iv"":""6ZPIEekizBjqJ1EbIZh7iw=="",...",15398116,3880633,texter,258743274,2022-12-30 13:00:57,Thank you. I'm looking at them right now. I re...
35808,8026068,2022-12-30 11:16:14,"aes256cbc#v0#{""iv"":""6ZPIEekizBjqJ1EbIZh7iw=="",...",15398116,3880633,texter,258743332,2022-12-30 13:02:22,I am as well.
35809,8026068,2022-12-30 11:16:14,"aes256cbc#v0#{""iv"":""6ZPIEekizBjqJ1EbIZh7iw=="",...",15398116,3880633,texter,258743391,2022-12-30 13:05:04,Thank you for making sure. We can close this c...
35810,8026068,2022-12-30 11:16:14,"aes256cbc#v0#{""iv"":""6ZPIEekizBjqJ1EbIZh7iw=="",...",15398116,3880633,texter,258743439,2022-12-30 13:06:36,Liked “Thank you! I really admire your strengt...


In [ ]:
df = df_messages_subsample[['conversation_id','message']].groupby(['conversation_id'])['message'].transform(lambda x: '. '.join(x))


df

0                                                   SURVEY
1          I want to continue the talk I was having before
2        I was literally peed on during school and othe...
3                               I need somebody to talk to
4        I'm thinking that at my school I'm just going ...
                               ...                        
35807    Thank you. I'm looking at them right now. I re...
35808                                        I am as well.
35809    Thank you for making sure. We can close this c...
35810    Liked “Thank you! I really admire your strengt...
35811                   Thank you and have a wonderful day
Name: message, Length: 31130, dtype: object

In [ ]:
convo_messages = {}

for convo_id in df_messages_subsample['conversation_id'].unique():
    df_messages_subsample_convo_i = df_messages_subsample[df_messages_subsample['conversation_id']== convo_id]
    messages = df_messages_subsample_convo_i['message'].tolist()
    # [print(n)  for n in messages if 'scrubbed' in n]
    messages = '. '.join([n.strip().strip('.').replace('[scrubbed]', 'Noah').replace('{{PHONE}}', '555-857-2939') for n in messages]) #unisex name
    convo_messages[convo_id] = messages

In [ ]:
df_metadata_subsample['message'] = df_metadata_subsample['conversation_id'].map(convo_messages)

df_metadata_subsample

,conversation_id,engaged,conversation_start_time_utc,addedToQueue,takenFromQueue,areacode,type,carrier,prank_ban,mandatory_report,...,convo_hour,abuse_unspecified,racism,immigration,out_of_country,work_financial,anxiety_stress,eating_body_image,gender_sexual_identity,message
0,2101698,1,2018-06-22 15:34:21,2018-06-22 15:35:31,2018-06-22 15:35:36,219.0,SMS,AT&T Wireless,0,0,...,15,0.0,NaN,NaN,NaN,0,1,0,0,TALK. High stress from my ex...he is lying to ...
1,2153727,1,2018-07-11 09:52:53,2018-07-11 09:53:33,2018-07-11 11:16:16,435.0,SMS,AT&T Wireless,0,0,...,9,0.0,NaN,NaN,NaN,0,0,0,0,I feel numb and hopeless. I go by Noah. I’m go...
2,7707947,1,2022-10-03 20:07:28,2022-10-03 20:09:08,2022-10-03 20:09:10,419.0,SMS,Verizon Wireless,0,0,...,20,0.0,0.0,0.0,0.0,0,0,0,0,Ended an abusive relationship and having dark ...
3,2260730,1,2018-08-18 23:13:58,2018-08-18 23:15:17,2018-08-18 23:35:41,410.0,MobileMessenger,AT&T Wireless,0,0,...,23,0.0,NaN,NaN,NaN,0,0,0,0,My husband is an alcoholic and mentally and ph...
4,2393094,1,2018-10-01 18:45:57,2018-10-01 18:46:58,2018-10-01 19:05:47,607.0,SMS,Verizon Wireless,0,0,...,18,0.0,NaN,NaN,NaN,0,0,0,0,Hello there. I think I might be in a situation...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1295,6789807,1,2022-01-23 02:28:22,2022-01-23 02:29:03,2022-01-23 02:38:50,NaN,Facebook,NaN,0,0,...,2,0.0,0.0,0.0,0.0,0,0,0,0,My wife wants to break up. I left my country t...
1296,1564711,1,2017-12-12 07:22:21,2017-12-12 07:22:38,2017-12-12 07:28:37,919.0,MobileMessenger,Verizon Wireless,0,0,...,7,NaN,NaN,NaN,NaN,0,0,0,0,I'm done. please hurry. I can't take this anym...
1297,2745382,1,2018-12-26 23:20:16,2018-12-26 23:20:51,2018-12-26 23:22:35,229.0,SMS,Cricket Wireless - ATT - SVR,0,0,...,23,0.0,NaN,NaN,NaN,0,0,0,0,"Family issues, suicidal. Okay. Hi, Noah. I'm N..."
1298,1469602,1,2017-11-14 06:16:27,2017-11-14 06:16:44,2017-11-14 06:17:01,707.0,SMS,Verizon Wireless,0,0,...,6,NaN,NaN,NaN,NaN,0,0,0,0,"I just wanna die, and not live anymore. I’ve b..."


In [ ]:
df_metadata_subsample.columns

Index(['conversation_id', 'engaged', 'conversation_start_time_utc',
       'addedToQueue', 'takenFromQueue', 'areacode', 'type', 'carrier',
       'prank_ban', 'mandatory_report', 'risk', 'suicidality',
       'suicidality_sorted', 'ir_flag', 'active_rescue', 'active_rescue2',
       'value', 'tags', 'tags_sorted', '3rd_party', 'abuse_emotional',
       'abuse_physical', 'abuse_sexual', 'bereavement', 'bully', 'covid_19',
       'depressed', 'domestic_violence', 'election', 'friend', 'homeless',
       'homicide', 'isolated', 'medical', 'medication', 'military', 'none',
       'other', 'prank', 'relationship', 'self_harm', 'substance', 'suicide',
       'testing', 'suicidal_capability', 'suicidal_desire', 'suicidal_intent',
       'timeframe', 'perceived_risk', 'convo_year', 'convo_month',
       'convo_dayofweek', 'convo_hour', 'abuse_unspecified', 'racism',
       'immigration', 'out_of_country', 'work_financial', 'anxiety_stress',
       'eating_body_image', 'gender_sexual_identity'

In [ ]:
df_metadata_subsample.to_csv(
    input_dir+f'datasets/train10_train_concurrent_metadata_100perconstruct_with_messages.csv')


In [ ]:
input_dir+f'datasets/train10_train_concurrent_metadata_100perconstruct_with_messages.csv'

'./../../../data/ctl/input/datasets/train10_train_concurrent_metadata_100perconstruct_with_messages.csv'

### Build small dataset

In [ ]:
import pandas as pd

In [ ]:
input_dir = './../../../data/ctl/input/'

In [ ]:
conversation_cols = [
    'conversation_id',
    'conversation_start_time_utc',
 'conversation_end_time_utc',
 'post_conversation_survey_end_time',
 'priority',
 'priority_predicted',
 'presenting_concern_message_id',
 'is_risk_assessed',
 'keyword',
 'num_wait_time_notifications',
 'language']

conversation_cols_to_drop = [e for e in conversation_cols if e not in ['conversation_id',
 'conversation_start_time_utc']]

In [ ]:
input_dir

'./../../../data/ctl/input/'

In [ ]:
df_metadata = pd.read_csv(input_dir+f'datasets/train10_train_concurrent_metadata.csv', index_col = 0)

df_messages = pd.read_parquet(input_dir+f'datasets/train10_train_concurrent_messages_texter.gzip', engine='pyarrow')        

In [ ]:
for message in message:
    

In [ ]:
%%time 
year = 2017
df_year = pd.read_parquet(input_dir+f'raw_data/messages_{year}.gzip', engine='pyarrow')        

CPU times: user 26.7 s, sys: 34.7 s, total: 1min 1s
Wall time: 1min 8s


In [ ]:
df_year.head()

,conversation_id,conversation_start_time_utc,conversation_end_time_utc,post_conversation_survey_end_time,priority,priority_predicted,presenting_concern_message_id,is_risk_assessed,keyword,num_wait_time_notifications,language,supervisor_notes,id,actor_id,interaction,message_id,message_timestamp_utc,message
0,700049,2017-01-01 00:00:22,2017-01-01 01:19:04,2017-01-01 05:23:43,2,2.0,None,0,Nami,0,en,None,1549009,283132,texter,28225179,2017-01-01 00:00:22,[scrubbed]
1,700049,2017-01-01 00:00:22,2017-01-01 01:19:04,2017-01-01 05:23:43,2,2.0,None,0,Nami,0,en,None,1549009,283132,texter,28225305,2017-01-01 00:02:55,My urge is so strong to trap my mom and [scrub...
2,700049,2017-01-01 00:00:22,2017-01-01 01:19:04,2017-01-01 05:23:43,2,2.0,None,0,Nami,0,en,None,1549012,96980,counselor,28225339,2017-01-01 00:03:42,"Hi my name is [scrubbed], may I have your name?"
3,700049,2017-01-01 00:00:22,2017-01-01 01:19:04,2017-01-01 05:23:43,2,2.0,None,0,Nami,0,en,None,1549009,283132,texter,28225362,2017-01-01 00:04:03,[scrubbed]
4,700049,2017-01-01 00:00:22,2017-01-01 01:19:04,2017-01-01 05:23:43,2,2.0,None,0,Nami,0,en,None,1549012,96980,counselor,28225394,2017-01-01 00:04:38,"Hi [scrubbed], what Is on your mind tonight?"


In [ ]:
for n in cols_to_drop:
    display(df_year[n].value_counts())

0    26442072
Name: is_risk_assessed, dtype: int64

In [ ]:
concurrent_df.columns

Index(['conversation_id', 'engaged', 'conversation_start_time_utc',
       'addedToQueue', 'takenFromQueue', 'areacode', 'type', 'carrier',
       'prank_ban', 'mandatory_report', 'risk', 'suicidality',
       'suicidality_sorted', 'ir_flag', 'active_rescue', 'active_rescue2',
       'value', 'tags', 'tags_sorted', '3rd_party', 'abuse_emotional',
       'abuse_physical', 'abuse_sexual', 'bereavement', 'bully', 'covid_19',
       'depressed', 'domestic_violence', 'election', 'friend', 'homeless',
       'homicide', 'isolated', 'medical', 'medication', 'military', 'none',
       'other', 'prank', 'relationship', 'self_harm', 'substance', 'suicide',
       'testing', 'suicidal_capability', 'suicidal_desire', 'suicidal_intent',
       'timeframe', 'perceived_risk', 'convo_year', 'convo_month',
       'convo_dayofweek', 'convo_hour', 'abuse_unspecified', 'racism',
       'immigration', 'out_of_country', 'work_financial', 'anxiety_stress',
       'eating_body_image', 'gender_sexual_identity'

In [ ]:
%%time
# create training DF from convo IDs

run_this = True


if run_this:

    df_set = []

    for year in [2017,2018,2019,2022]:
        print(year)
        d_year_ids = d.get(year)
        a
        # df_year.to_parquet(input_dir+f'messages_{year}.gzip', compression = 'gzip', engine='pyarrow')        

        df_year_set = df_year[df_year['conversation_id'].isin(d_year_ids)]
        df_set.append(df_year_set)


    # df_set.to_csv(input_dir+f'../datasets/train10_train_messages.csv')

2017


NameError: name 'd' is not defined

In [ ]:
df_set = pd.concat(df_set)

In [ ]:
    df_set = pd.concat(df_set)
    df_set = df_set.reset_index(drop=True)    
    os.makedirs(input_dir+f'../datasets', exist_ok=True)
    df_set.to_parquet(input_dir+f'../datasets/train10_train_messages.gzip', compression = 'gzip', engine='pyarrow')        

In [ ]:
df_set

NameError: name 'df_set' is not defined

In [ ]:
df_set

NameError: name 'df_set' is not defined

In [ ]:
df_set

(7506103, 18)

### Load

In [ ]:
input_dir = './../../../data/ctl/input/'

In [ ]:
metadata_df = pd.read_csv(input_dir+'datasets/train10_train_metadata.csv', index_col = 0)

In [ ]:

# messages_df = pd.read_csv(input_dir+'../datasets/train10_train_messages.csv', index_col = 0, lineterminator='\n')


In [ ]:
messages_df

,conversation_id,conversation_start_time_utc,conversation_end_time_utc,post_conversation_survey_end_time,priority,priority_predicted,presenting_concern_message_id,is_risk_assessed,keyword,num_wait_time_notifications,language,supervisor_notes,id,actor_id,interaction,message_id,message_timestamp_utc,message
0,700061,2017-01-01 00:12:12,2017-01-01 00:29:14,NaN,2,2.0,NaN,0,Go,0,en,NaN,1549031,331196,texter,28225720,2017-01-01 00:12:12,Go
1,700061,2017-01-01 00:12:12,2017-01-01 00:29:14,NaN,2,2.0,NaN,0,Go,0,en,NaN,1549031,331196,texter,28225758,2017-01-01 00:12:54,I feel swamped with marriage like I can't hand...
2,700061,2017-01-01 00:12:12,2017-01-01 00:29:14,NaN,2,2.0,NaN,0,Go,0,en,NaN,1549036,276284,counselor,28225806,2017-01-01 00:14:06,"Hi, i'm [scrubbed]. You sound upset. Can yo..."
3,700061,2017-01-01 00:12:12,2017-01-01 00:29:14,NaN,2,2.0,NaN,0,Go,0,en,NaN,1549031,331196,texter,28225940,2017-01-01 00:16:52,"It started this early afternoon, with my husba..."
4,700061,2017-01-01 00:12:12,2017-01-01 00:29:14,NaN,2,2.0,NaN,0,Go,0,en,NaN,1549036,276284,counselor,28225990,2017-01-01 00:17:54,So you and your husband had an argument that s...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7506098,8031232,2022-12-31 23:58:09,2023-01-01 00:14:12,NaN,4,4.0,258893299.0,0,HOME,0,en,"aes256cbc#v0#{""iv"":""ADEwAnL3a2a8FKkpJo1Ljg=="",...",15407359,1403931,texter,258893708,2023-01-01 00:13:32,My family has making me feel like I'm not im...
7506099,8031232,2022-12-31 23:58:09,2023-01-01 00:14:12,NaN,4,4.0,258893299.0,0,HOME,0,en,"aes256cbc#v0#{""iv"":""ADEwAnL3a2a8FKkpJo1Ljg=="",...",15407359,1403931,texter,258893724,2023-01-01 00:13:41,Stop
7506100,8031234,2022-12-31 23:58:12,2023-01-01 00:59:21,NaN,2,2.0,258893331.0,0,Help,3,en,"aes256cbc#v0#{""iv"":""LJAOh+FsV0FzFq+0vLLcXg=="",...",15407361,2795149,texter,258892475,2022-12-31 23:58:12,Help
7506101,8031234,2022-12-31 23:58:12,2023-01-01 00:59:21,NaN,2,2.0,258893331.0,0,Help,3,en,"aes256cbc#v0#{""iv"":""LJAOh+FsV0FzFq+0vLLcXg=="",...",15407361,2795149,texter,258893331,2023-01-01 00:08:56,I have been crying uncontrollably for 2 days! ...


In [ ]:
messages_df['conversation_id'].unique().shape

(286387,)

In [ ]:
metadata_df.shape

(286585, 61)

In [ ]:
concurrent_df[concurrent_df['isolated']==1]['conversation_id'].values

array([2908377, 2288791, 6916203, ..., 3934572, 3041199, 2141261])

# OLD

In [ ]:
metadata_i.columns

Index(['conversation_id', 'engaged', 'conversation_start_time_utc',
       'addedToQueue', 'takenFromQueue', 'areacode', 'type', 'carrier',
       'prank_ban', 'mandatory_report', 'risk', 'suicidality',
       'suicidality_sorted', 'ir_flag', 'active_rescue', 'active_rescue2',
       'value', 'tags', 'tags_sorted', '3rd_party', 'abuse', 'abuse_emotional',
       'abuse_physical', 'abuse_sexual', 'anger', 'anxiety', 'bereavement',
       'bully', 'depressed', 'domestic_violence', 'eating', 'election',
       'friend', 'gender', 'homeless', 'housing', 'hunger', 'isolated',
       'lgbtq', 'medical', 'military', 'none', 'other', 'prank', 'pregnancy',
       'relationship', 'self_harm', 'sexual_abuse', 'stress_anxiety',
       'substance', 'suicide', 'testing', 'suicidal_capability',
       'suicidal_desire', 'suicidal_intent', 'timeframe', 'perceived_risk',
       'convo_year', 'convo_month', 'convo_dayofweek', 'convo_hour'],
      dtype='object')

In [ ]:
binary_columns = ['ir_flag', 'active_rescue','abuse', 'abuse_emotional',
       'abuse_physical', 'abuse_sexual', 'anger', 'anxiety', 'bereavement',
       'bully', 'depressed', 'domestic_violence', 'eating', 'election',
       'friend', 'gender', 'homeless', 'housing', 'hunger', 'isolated',
       'lgbtq', 'medical', 'military', 'none', 'other', 'prank', 'pregnancy',
       'relationship', 'self_harm', 'sexual_abuse', 'stress_anxiety',
       'substance', 'suicide', 'testing', 'suicidal_capability',
       'suicidal_desire', 'suicidal_intent', 'timeframe']

In [ ]:
messages_i['actor_id'].value_counts()

1638462    9675
3828729    6784
3815253    5287
3655150    4772
3033737    4218
           ... 
3811303       1
3827361       1
3817444       1
3825945       1
3807567       1
Name: actor_id, Length: 96597, dtype: int64

In [ ]:
metadata_i['active_rescue2'].shape

(1242398,)

In [ ]:

(metadata_i['ir_flag']==metadata_i['active_rescue2']).sum()/metadata_i.shape[0]

0.9901464748011507

In [ ]:
metadata_i[metadata_i['active_rescue']==1]

,conversation_id,engaged,conversation_start_time_utc,addedToQueue,takenFromQueue,areacode,type,carrier,prank_ban,mandatory_report,risk,suicidality,suicidality_sorted,ir_flag,active_rescue,active_rescue2,value,tags,tags_sorted,3rd_party,abuse_emotional,abuse_physical,abuse_sexual,abuse_unspecified,anxiety,bereavement,bully,covid_19,depressed,eating,election,gender,immigration,isolated,none,other,out_of_country,prank,racism,relationship,self_harm,substance,suicide,testing,suicidal_capability,suicidal_desire,suicidal_intent,timeframe,perceived_risk,convo_year,convo_month,convo_dayofweek,convo_hour
134,6708002,1,2022-01-01 00:39:54,2022-01-01 00:40:57,2022-01-01 00:41:15,864,SMS,AT&T Wireless,0,0,"a:4:{i:0;s:15:""suicidal_desire"";i:1;s:15:""suic...","[suicidal_desire, suicidal_intent, suicidal_ca...","[suicidal_capability, suicidal_desire, suicida...",0,1,0,"a:1:{i:0;s:7:""suicide"";}",[suicide],[suicide],0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,1,1,1,1,4,2022,1,5,0
226,6708094,1,2022-01-01 01:04:22,2022-01-01 01:06:02,2022-01-01 01:06:25,NA,Whatsapp,Google (Grand Central) - SVR,0,0,"a:4:{i:0;s:15:""suicidal_desire"";i:1;s:15:""suic...","[suicidal_desire, suicidal_intent, suicidal_ca...","[suicidal_capability, suicidal_desire, suicida...",0,1,0,"a:1:{i:0;s:7:""suicide"";}",[suicide],[suicide],0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,1,1,1,1,4,2022,1,5,1
329,6708198,1,2022-01-01 01:35:37,2022-01-01 01:36:03,2022-01-01 01:36:50,573,SMS,AT&T Wireless,0,0,"a:4:{i:0;s:15:""suicidal_desire"";i:1;s:15:""suic...","[suicidal_desire, suicidal_intent, suicidal_ca...","[suicidal_capability, suicidal_desire, suicida...",0,1,0,"a:2:{i:0;s:7:""suicide"";i:1;s:12:""relationship"";}","[suicide, relationship]","[relationship, suicide]",0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,1,0,1,1,1,1,4,2022,1,5,1
1181,6709060,1,2022-01-01 05:21:33,2022-01-01 05:21:46,2022-01-01 05:51:20,573,SMS,"T-Mobile USA, Inc.",0,0,"a:4:{i:0;s:15:""suicidal_desire"";i:1;s:15:""suic...","[suicidal_desire, suicidal_intent, suicidal_ca...","[suicidal_capability, suicidal_desire, suicida...",0,1,0,"a:1:{i:0;s:7:""suicide"";}",[suicide],[suicide],0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,1,1,1,1,4,2022,1,5,5
1331,6709211,1,2022-01-01 06:00:24,2022-01-01 06:00:39,2022-01-01 06:02:05,NA,Facebook,None,0,0,"a:4:{i:0;s:15:""suicidal_desire"";i:1;s:15:""suic...","[suicidal_desire, suicidal_intent, suicidal_ca...","[suicidal_capability, suicidal_desire, suicida...",0,1,0,"a:1:{i:0;s:7:""suicide"";}",[suicide],[suicide],0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,1,1,1,1,4,2022,1,5,6
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1241732,8030479,1,2022-12-31 18:42:09,2022-12-31 18:42:31,2022-12-31 18:42:35,214,SMS,T-Mobile,0,0,"a:4:{i:0;s:15:""suicidal_desire"";i:1;s:15:""suic...","[suicidal_desire, suicidal_intent, suicidal_ca...","[suicidal_capability, suicidal_desire, suicida...",0,1,0,"a:1:{i:0;s:7:""suicide"";}",[suicide],[suicide],0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,1,1,1,1,4,2022,12,5,18
1241758,8030508,1,2022-12-31 18:55:38,2022-12-31 18:56:09,2022-12-31 18:56:10,484,SMS,Verizon Wireless,0,0,"a:4:{i:0;s:15:""suicidal_desire"";i:1;s:15:""suic...","[suicidal_desire, suicidal_intent, suicidal_ca...","[suicidal_capability, suicidal_desire, suicida...",0,1,0,"a:1:{i:0;s:7:""suicide"";}",[suicide],[suicide],0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,1,1,1,1,4,2022,12,5,18
1241820,8030589,1,2022-12-31 19:32:30,2022-12-31 19:34:40,2022-12-31 19:35:35,NA,Webchat,None,0,0,"a:4:{i:0;s:15:""suicidal_desire"";i:1;s:15:""suic...","[suicidal_desire, suicidal_intent, suicidal_ca...","[suicidal_capability, suicidal_desire, suicida...",0,1,0,"a:1:{i:0;s:7:""suicide"";}",[suicide],[suicide],0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,1,1,1,1,4,2022,12,5,19
1241994,803

In [ ]:
year_month = '2022-12'
messages_i = pd.read_parquet(input_dir+f'messages_{year_month}.gzip', engine='pyarrow')        

In [ ]:
for i in [8030479,8030508, 8030786, 8031145]:
    print(i)
    print(messages_i[messages_i['conversation_id']==i][['interaction', 'message']].values)

8030479
[['texter' 'TikTok']
 ['texter' 'Suicidal ']
 ['counselor'
  "Hi there, my name is [scrubbed], thank you for texting today. It took strength to reach out, and I'm here for you."]
 ['counselor'
  'Before we talk a little more, is there a name or nickname you prefer to go by?']
 ['texter' 'Hi']
 ['texter' "I'd like to go by [scrubbed] "]
 ['counselor' 'It is very nice to meet you [scrubbed]. ']
 ['texter' 'Nice to meet you too']
 ['counselor'
  "It sounds like you're experiencing a lot of pain today, would you mind telling me a little more about what led you to text? "]
 ['texter' "The weight that's on me from people"]
 ['counselor'
  'The pressures we feel from others for so many reasons can absolutely feel like it is pressing in on us. Would you tell me more about what you mean? ']
 ['texter'
  'So I like this [scrubbed] [scrubbed], [scrubbed] Minaj, and she has a fanbase called Barbz. We are the strongest and the most protective people to keep [scrubbed] strong. But [scrubbed]

In [ ]:
tag_cols = ['abuse', 'abuse_emotional',
       'abuse_physical', 'abuse_sexual', 'anger', 'anxiety', 'bereavement',
       'bully', 'depressed', 'domestic_violence', 'eating', 'election',
       'friend', 'gender', 'homeless', 'housing', 'hunger', 'isolated',
       'lgbtq', 'medical', 'military', 'none', 'other', 'prank', 'pregnancy',
       'relationship', 'self_harm', 'sexual_abuse', 'stress_anxiety',
       'substance', 'suicide', 'testing', 'suicidal_capability',
       'suicidal_desire', 'suicidal_intent', 'timeframe', 'perceived_risk']

In [ ]:
print(metadata_i.shape)
print(metadata_i[metadata_i['engaged']==0].shape)

(420905, 61)
(99158, 61)


In [ ]:
metadata_i[metadata_i['engaged']==0][tag_cols].sum()

abuse                      24
abuse_emotional            19
abuse_physical              7
abuse_sexual               16
anger                      39
anxiety                    82
bereavement                46
bully                      48
depressed                 589
domestic_violence           3
eating                     81
election                   98
friend                    145
gender                     24
homeless                    9
housing                    19
hunger                      2
isolated                  116
lgbtq                      40
medical                    46
military                    1
none                    34092
other                    2413
prank                     256
pregnancy                  22
relationship              458
self_harm                 226
sexual_abuse               25
stress_anxiety            342
substance                  49
suicide                   645
testing                  1967
suicidal_capability        13
suicidal_d

In [ ]:
metadata_i[(metadata_i['engaged']==0) & (metadata_i['relationship']==1)]

,conversation_id,engaged,conversation_start_time_utc,addedToQueue,takenFromQueue,areacode,type,carrier,prank_ban,mandatory_report,risk,suicidality,suicidality_sorted,ir_flag,active_rescue,active_rescue2,value,tags,tags_sorted,3rd_party,abuse,abuse_emotional,abuse_physical,abuse_sexual,anger,anxiety,bereavement,bully,depressed,domestic_violence,eating,election,friend,gender,homeless,housing,hunger,isolated,lgbtq,medical,military,none,other,prank,pregnancy,relationship,self_harm,sexual_abuse,stress_anxiety,substance,suicide,testing,suicidal_capability,suicidal_desire,suicidal_intent,timeframe,perceived_risk,convo_year,convo_month,convo_dayofweek,convo_hour
61458,336972,0,2016-03-07 05:46:17,2016-03-07 05:47:08,2016-03-07 06:27:38,330,MobileMessenger,Verizon Wireless,0,0,None,[],[],0,0,0,"a:1:{i:0;s:12:""relationship"";}",[relationship],[relationship],0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,2,2016,3,0,5
64341,339895,0,2016-03-10 02:34:25,2016-03-10 02:34:39,2016-03-10 02:34:55,304,MobileMessenger,US Cellular Corp.,0,0,None,[],[],0,0,0,"a:2:{i:0;s:9:""depressed"";i:1;s:12:""relationshi...","[depressed, relationship]","[depressed, relationship]",0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,2,2016,3,3,2
64749,340309,0,2016-03-10 11:40:19,2016-03-10 11:41:11,2016-03-10 11:41:16,270,MobileMessenger,Bluegrass Cellular,0,0,None,[],[],0,0,0,"a:2:{i:0;s:4:""none"";i:1;s:12:""relationship"";}","[none, relationship]","[none, relationship]",0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,2,2016,3,3,11
66162,341735,0,2016-03-12 00:13:07,2016-03-12 00:16:09,2016-03-12 00:24:39,408,MobileMessenger,Sprint,0,0,None,[],[],0,0,0,"a:2:{i:0;s:4:""none"";i:1;s:12:""relationship"";}","[none, relationship]","[none, relationship]",0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,2,2016,3,5,0
66698,342280,0,2016-03-12 14:58:43,2016-03-12 14:59:33,2016-03-12 15:00:55,210,MobileMessenger,AT&T (Cingular Orange),0,0,None,[],[],0,0,0,"a:2:{i:0;s:12:""relationship"";i:1;s:6:""friend"";}","[relationship, friend]","[friend, relationship]",0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,2,2016,3,5,14
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
416462,695556,0,2016-12-28 07:37:36,2016-12-28 07:39:36,2016-12-28 07:55:59,443,SMS,Verizon Wireless,0,0,None,[],[],0,0,0,"a:2:{i:0;s:4:""none"";i:1;s:12:""relationship"";}","[none, relationship]","[none, relationship]",0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,2,2016,12,2,7
417629,696736,0,2016-12-29 05:34:46,2016-12-29 05:36:14,2016-12-29 05:36:19,304,MobileMessenger,AT&T Wireless,0,0,None,[],[],0,0,0,"a:1:{i:0;s:12:""relationship"";}",[relationship],[relationship],0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,2,2016,12,3,5
417879,696989,0,2016-12-29 08:13:46,2016-12-29 08:15:27,2016-12-29 08:20:29,480,SMS,"T-Mobile USA, Inc.",0,0,None,[],[],0,0,0,"a:1:{i:0;s:12:""relationship"";}",[relationship],[relationship],0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,2,2016,12,3,8
419057,698178,0,2016-12-30 06:52:04,2016-12-30 06:52:20,2016-12-30 06:58:34,415,SMS,AT&T Wireless,0,0,None,[],[],0,0,0,"a:1:{i:0;s:12:""relationship"";}",[relationship],[relationship],0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,2,2016,12,4,6


In [ ]:
for i in [697402,697657, 698727, 683578, 700002]:
    print(i)
    print(messages_i[messages_i['conversation_id']==i][['interaction', 'message']].values)

697402
[['texter'
  "Hello, send police and ambulance to 11579 Clear [scrubbed] rd. [scrubbed], PA 15537. I don't want to be found by my family! Im in the garage."]
 ['counselor'
  "Hi there, I'm here. Can I ask if you are able to call 911 yourself right now?"]
 ['counselor' 'Are you able to talk right now?']
 ['counselor' "We are going to call 911. I'm here if you want to talk."]
 ['counselor'
  "We have contacted 911 and they should be on their way. I'm still here if you want to talk."]
 ['counselor'
  "I do hope you're safe. 911 should be there soon. We are here 24/7 if you ever need to talk. "]]
697657
[['texter'
  "Hey I'm not sure how this works . Not sure if I have to be absolutely suicidal to text this hotline .."]
 ['counselor'
  "Hey, my name is [scrubbed]. You don't necessarily have to be suicidal to text in! We provide crisis counseling for people dealing with lots of issues.What's going on?"]
 ['counselor'
  "It's been a few minutes since your last message; if you still ne

In [ ]:
for year_month in year_months:
    messages_i = pd.read_parquet(input_dir+f'messages_{year_month}.gzip', engine='pyarrow')        
    break

In [ ]:
messages_i.sum()

In [ ]:
for i in [700001,700005, 700009]:
    print(i)
    print(messages_i[messages_i['conversation_id']==i][['interaction', 'message']].values)

700001
[['texter' 'Start']
 ['counselor'
  "Hello, thank you for texting Crisis Text Line. My name is [scrubbed] and I'm here to listen to anything on your mind. Can you tell me what's going on?"]
 ['counselor'
  "I'm sorry for your wait, but I'm here to listen now. Are you still there?"]
 ['counselor'
  "If I don't get a response from you in the next 5 minutes, I'll have to close the conversation. "]
 ['counselor'
  'We are here 24/7 if you need additional support. Take care of yourself.']]
700005
[['texter' 'HOPELINE']
 ['texter' 'STOP']]
700009
[['texter' 'What is this ']
 ['texter' 'I need a distraction ']
 ['counselor'
  "Hi, my name is [scrubbed] and I'm a counselor here. How are you doing tonight?"]
 ['counselor'
  "Are you there? Let me know and I'll be happy to talk to you!"]]


In [ ]:
year_month = '2016-04'
messages_i = pd.read_parquet(input_dir+f'messages_{year_month}.gzip', engine='pyarrow')        

In [ ]:
year_month = '2017-01'
messages_17 = pd.read_parquet(input_dir+f'messages_{year_month}.gzip', engine='pyarrow')        

In [ ]:
messages_17

,conversation_id,conversation_start_time_utc,conversation_end_time_utc,post_conversation_survey_end_time,priority,priority_predicted,presenting_concern_message_id,is_risk_assessed,keyword,num_wait_time_notifications,language,supervisor_notes,id,actor_id,interaction,message_id,message_timestamp_utc,message
0,700049,2017-01-01 00:00:22,2017-01-01 01:19:04,2017-01-01 05:23:43,2,2.0,None,0,Nami,0,en,None,1549009,283132,texter,28225179,2017-01-01 00:00:22,[scrubbed]
1,700049,2017-01-01 00:00:22,2017-01-01 01:19:04,2017-01-01 05:23:43,2,2.0,None,0,Nami,0,en,None,1549009,283132,texter,28225305,2017-01-01 00:02:55,My urge is so strong to trap my mom and [scrub...
49,700049,2017-01-01 00:00:22,2017-01-01 01:19:04,2017-01-01 05:23:43,2,2.0,None,0,Nami,0,en,None,1549012,96980,counselor,28225339,2017-01-01 00:03:42,"Hi my name is [scrubbed], may I have your name?"
2,700049,2017-01-01 00:00:22,2017-01-01 01:19:04,2017-01-01 05:23:43,2,2.0,None,0,Nami,0,en,None,1549009,283132,texter,28225362,2017-01-01 00:04:03,[scrubbed]
50,700049,2017-01-01 00:00:22,2017-01-01 01:19:04,2017-01-01 05:23:43,2,2.0,None,0,Nami,0,en,None,1549012,96980,counselor,28225394,2017-01-01 00:04:38,"Hi [scrubbed], what Is on your mind tonight?"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1680295,746741,2017-01-31 23:59:36,2017-02-01 01:33:19,2017-02-01 11:57:48,4,3.0,None,0,Help,0,en,None,1652353,352918,texter,30037648,2017-02-01 01:17:55,I'm feeling a little better talking to someone...
1680312,746741,2017-01-31 23:59:36,2017-02-01 01:33:19,2017-02-01 11:57:48,4,3.0,None,0,Help,0,en,None,1652355,336859,counselor,30037869,2017-02-01 01:21:04,You said that you're unsure of where to begin ...
1680313,746741,2017-01-31 23:59:36,2017-02-01 01:33:19,2017-02-01 11:57:48,4,3.0,None,0,Help,0,en,None,1652355,336859,counselor,30037890,2017-02-01 01:21:20,Is there anything else you wanted to say befor...
1680314,746741,2017-01-31 23:59:36,2017-02-01 01:33:19,2017-02-01 11:57:48,4,3.0,None,0,Help,0,en,None,1652355,336859,counselor,30038723,2017-02-01 01:33:05,"I'm going to close the conversation now, and I..."


In [ ]:
cols = ['conversation_id', 'actor_id']

In [ ]:
%%time
convo_actor_ids = {}


for convo_id in messages_i['conversation_id'].values:
    convo_actor_ids[convo_id] = len(messages_i[messages_i['conversation_id'] == convo_id]['actor_id'].unique())
    
    
    
    


KeyboardInterrupt: 

In [ ]:
pd.DataFrame(convo_actor_ids)

In [ ]:


messages_i.groupby(cols)['actor_id'].count()

conversation_id  actor_id
360678           152258       6
                 161573       6
360679           152761      15
                 153209       3
                 161557      20
                             ..
393030           169849      41
393183           66279       13
                 96930       22
                 121712      38
                 175952      82
Name: actor_id, Length: 62877, dtype: int64

In [ ]:
print(messages_i[messages_i['conversation_id']==360679]['id'].unique())
messages_i[messages_i['conversation_id']==360679]['id']

[834307 834308 834315]


12    834307
13    834307
32    834308
14    834307
33    834308
15    834307
34    834308
16    834307
17    834307
35    834315
18    834307
36    834315
19    834307
20    834307
37    834315
21    834307
38    834315
22    834307
23    834307
39    834315
24    834307
40    834315
25    834307
41    834315
26    834307
42    834315
27    834307
43    834315
28    834307
44    834315
45    834315
46    834315
29    834307
47    834315
30    834307
48    834315
31    834307
49    834315
Name: id, dtype: int64

In [ ]:
messages_i[messages_i['conversation_id']==360679]['message'].values

array(['SURVEY', 'Ok',
       'Hello and thanks for reaching out to CTL today.  My name is [scrubbed]- would you like to share your name?',
       'Hi my name is [scrubbed]',
       "Thanks, [scrubbed].  What's on your mind?", 'Depression',
       "Sounds like you're going through a lot right now.  Could you tell me more about your depression?  ",
       "I don't know I've been crying for two days and I feel like doing something bad",
       'Hello are you still there ?',
       'That can be intense to feel like doing something bad. Would you like to tell me more about that [scrubbed]? ',
       "Well I'm 12 years old",
       'This is a safe space to text about anything. You are very brave to text in tonight [scrubbed].',
       "And I'm very emotional and I get in my little emotional zone really fast if someone brings me down",
       "I don't know why I get like this please help me !",
       'You mentioned going into your emotional zone when someone brings you down. How has your em

In [ ]:
messages_i

,conversation_id,conversation_start_time_utc,conversation_end_time_utc,post_conversation_survey_end_time,priority,priority_predicted,presenting_concern_message_id,is_risk_assessed,keyword,num_wait_time_notifications,language,supervisor_notes,id,actor_id,interaction,message_id,message_timestamp_utc,message
0,360678,2016-04-01 00:04:53,2016-04-01 00:20:44,2016-04-01 10:55:56,2,2.0,None,0,Hi,0,en,None,834304,161573,texter,15331251,2016-04-01 00:04:55,Hi Tiger
1,360678,2016-04-01 00:04:53,2016-04-01 00:20:44,2016-04-01 10:55:56,2,2.0,None,0,Hi,0,en,None,834304,161573,texter,15331283,2016-04-01 00:05:38,I'm really stressed
6,360678,2016-04-01 00:04:53,2016-04-01 00:20:44,2016-04-01 10:55:56,2,2.0,None,0,Hi,0,en,None,834306,152258,counselor,15331311,2016-04-01 00:06:13,"Hi, thank you for texting into the Crisis Text..."
2,360678,2016-04-01 00:04:53,2016-04-01 00:20:44,2016-04-01 10:55:56,2,2.0,None,0,Hi,0,en,None,834304,161573,texter,15331339,2016-04-01 00:06:43,No
7,360678,2016-04-01 00:04:53,2016-04-01 00:20:44,2016-04-01 10:55:56,2,2.0,None,0,Hi,0,en,None,834306,152258,counselor,15331354,2016-04-01 00:07:02,"That is completely fine, you do not have to sh..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1184826,392815,2016-04-30 23:59:43,2016-05-01 00:58:27,2016-05-01 10:56:06,2,2.0,None,0,Hel,0,en,None,901060,177476,texter,16659286,2016-05-01 00:37:32,That would be great. I would like that. Thank ...
1184841,392815,2016-04-30 23:59:43,2016-05-01 00:58:27,2016-05-01 10:56:06,2,2.0,None,0,Hel,0,en,None,901064,152358,counselor,16659306,2016-05-01 00:38:17,Here's the link: {{URL}}
1184827,392815,2016-04-30 23:59:43,2016-05-01 00:58:27,2016-05-01 10:56:06,2,2.0,None,0,Hel,0,en,None,901060,177476,texter,16659346,2016-05-01 00:39:40,Thank you so much
1184842,392815,2016-04-30 23:59:43,2016-05-01 00:58:27,2016-05-01 10:56:06,2,2.0,None,0,Hel,0,en,None,901064,152358,counselor,16659382,2016-05-01 00:40:47,"You're very welcome, [scrubbed]! I wish you th..."


In [ ]:
messages_17

,conversation_id,conversation_start_time_utc,conversation_end_time_utc,post_conversation_survey_end_time,priority,priority_predicted,presenting_concern_message_id,is_risk_assessed,keyword,num_wait_time_notifications,language,supervisor_notes,id,actor_id,interaction,message_id,message_timestamp_utc,message
0,700049,2017-01-01 00:00:22,2017-01-01 01:19:04,2017-01-01 05:23:43,2,2.0,None,0,Nami,0,en,None,1549009,283132,texter,28225179,2017-01-01 00:00:22,[scrubbed]
1,700049,2017-01-01 00:00:22,2017-01-01 01:19:04,2017-01-01 05:23:43,2,2.0,None,0,Nami,0,en,None,1549009,283132,texter,28225305,2017-01-01 00:02:55,My urge is so strong to trap my mom and [scrub...
49,700049,2017-01-01 00:00:22,2017-01-01 01:19:04,2017-01-01 05:23:43,2,2.0,None,0,Nami,0,en,None,1549012,96980,counselor,28225339,2017-01-01 00:03:42,"Hi my name is [scrubbed], may I have your name?"
2,700049,2017-01-01 00:00:22,2017-01-01 01:19:04,2017-01-01 05:23:43,2,2.0,None,0,Nami,0,en,None,1549009,283132,texter,28225362,2017-01-01 00:04:03,[scrubbed]
50,700049,2017-01-01 00:00:22,2017-01-01 01:19:04,2017-01-01 05:23:43,2,2.0,None,0,Nami,0,en,None,1549012,96980,counselor,28225394,2017-01-01 00:04:38,"Hi [scrubbed], what Is on your mind tonight?"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1680295,746741,2017-01-31 23:59:36,2017-02-01 01:33:19,2017-02-01 11:57:48,4,3.0,None,0,Help,0,en,None,1652353,352918,texter,30037648,2017-02-01 01:17:55,I'm feeling a little better talking to someone...
1680312,746741,2017-01-31 23:59:36,2017-02-01 01:33:19,2017-02-01 11:57:48,4,3.0,None,0,Help,0,en,None,1652355,336859,counselor,30037869,2017-02-01 01:21:04,You said that you're unsure of where to begin ...
1680313,746741,2017-01-31 23:59:36,2017-02-01 01:33:19,2017-02-01 11:57:48,4,3.0,None,0,Help,0,en,None,1652355,336859,counselor,30037890,2017-02-01 01:21:20,Is there anything else you wanted to say befor...
1680314,746741,2017-01-31 23:59:36,2017-02-01 01:33:19,2017-02-01 11:57:48,4,3.0,None,0,Help,0,en,None,1652355,336859,counselor,30038723,2017-02-01 01:33:05,"I'm going to close the conversation now, and I..."


In [ ]:
year_month = '2017-01'
messages_17 = pd.read_parquet(input_dir+f'messages_{year_month}.gzip', engine='pyarrow')        

In [ ]:
messages_17

,conversation_id,conversation_start_time_utc,conversation_end_time_utc,post_conversation_survey_end_time,priority,priority_predicted,presenting_concern_message_id,is_risk_assessed,keyword,num_wait_time_notifications,language,supervisor_notes,id,actor_id,interaction,message_id,message_timestamp_utc,message
0,700049,2017-01-01 00:00:22,2017-01-01 01:19:04,2017-01-01 05:23:43,2,2.0,None,0,Nami,0,en,None,1549009,283132,texter,28225179,2017-01-01 00:00:22,[scrubbed]
1,700049,2017-01-01 00:00:22,2017-01-01 01:19:04,2017-01-01 05:23:43,2,2.0,None,0,Nami,0,en,None,1549009,283132,texter,28225305,2017-01-01 00:02:55,My urge is so strong to trap my mom and [scrub...
49,700049,2017-01-01 00:00:22,2017-01-01 01:19:04,2017-01-01 05:23:43,2,2.0,None,0,Nami,0,en,None,1549012,96980,counselor,28225339,2017-01-01 00:03:42,"Hi my name is [scrubbed], may I have your name?"
2,700049,2017-01-01 00:00:22,2017-01-01 01:19:04,2017-01-01 05:23:43,2,2.0,None,0,Nami,0,en,None,1549009,283132,texter,28225362,2017-01-01 00:04:03,[scrubbed]
50,700049,2017-01-01 00:00:22,2017-01-01 01:19:04,2017-01-01 05:23:43,2,2.0,None,0,Nami,0,en,None,1549012,96980,counselor,28225394,2017-01-01 00:04:38,"Hi [scrubbed], what Is on your mind tonight?"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1680295,746741,2017-01-31 23:59:36,2017-02-01 01:33:19,2017-02-01 11:57:48,4,3.0,None,0,Help,0,en,None,1652353,352918,texter,30037648,2017-02-01 01:17:55,I'm feeling a little better talking to someone...
1680312,746741,2017-01-31 23:59:36,2017-02-01 01:33:19,2017-02-01 11:57:48,4,3.0,None,0,Help,0,en,None,1652355,336859,counselor,30037869,2017-02-01 01:21:04,You said that you're unsure of where to begin ...
1680313,746741,2017-01-31 23:59:36,2017-02-01 01:33:19,2017-02-01 11:57:48,4,3.0,None,0,Help,0,en,None,1652355,336859,counselor,30037890,2017-02-01 01:21:20,Is there anything else you wanted to say befor...
1680314,746741,2017-01-31 23:59:36,2017-02-01 01:33:19,2017-02-01 11:57:48,4,3.0,None,0,Help,0,en,None,1652355,336859,counselor,30038723,2017-02-01 01:33:05,"I'm going to close the conversation now, and I..."


In [ ]:
metadata_i[metadata_i['depressed']==1]

,conversation_id,engaged,conversation_start_time_utc,addedToQueue,takenFromQueue,areacode,type,carrier,prank_ban,mandatory_report,risk,suicidality,suicidality_sorted,ir_flag,active_rescue,active_rescue2,value,tags,tags_sorted,3rd_party,abuse,abuse_emotional,abuse_physical,abuse_sexual,anger,anxiety,bereavement,bully,depressed,domestic_violence,eating,election,friend,gender,homeless,housing,hunger,isolated,lgbtq,medical,military,none,other,prank,pregnancy,relationship,self_harm,sexual_abuse,stress_anxiety,substance,suicide,testing,suicidal_capability,suicidal_desire,suicidal_intent,timeframe,perceived_risk,convo_year,convo_month,convo_dayofweek,convo_hour
59511,334984,1,2016-03-05 08:40:25,2016-03-05 08:40:55,2016-03-05 08:49:25,510,MobileMessenger,AT&T Wireless,0,0,None,[],[],0,0,0,"a:2:{i:0;s:9:""depressed"";i:1;s:6:""friend"";}","[depressed, friend]","[depressed, friend]",0,0,0,0,0,0,0,0,0,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,2,2016,3,5,8
59515,334988,1,2016-03-05 08:45:05,2016-03-05 08:46:04,2016-03-05 08:57:09,330,MobileMessenger,Verizon Wireless,0,0,None,[],[],0,0,0,"a:1:{i:0;s:9:""depressed"";}",[depressed],[depressed],0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,2,2016,3,5,8
59517,334990,1,2016-03-05 08:54:24,2016-03-05 08:56:46,2016-03-05 08:59:43,765,MobileMessenger,Verizon Wireless,0,0,"s:15:""suicidal_desire"";",[suicidal_desire],[suicidal_desire],0,0,0,"a:5:{i:0;s:9:""depressed"";i:1;s:7:""suicide"";i:2...","[depressed, suicide, stress_anxiety, isolated,...","[depressed, isolated, mental, stress_anxiety, ...",0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,1,0,0,1,0,0,3,2016,3,5,8
59518,334991,1,2016-03-05 08:55:22,2016-03-05 08:57:55,2016-03-05 09:02:28,623,MobileMessenger,Verizon Wireless,0,0,"s:19:""suicidal_capability"";",[suicidal_capability],[suicidal_capability],0,0,0,"a:6:{i:0;s:9:""depressed"";i:1;s:7:""suicide"";i:2...","[depressed, suicide, family, isolated, substan...","[depressed, family, isolated, sexual_abuse, su...",0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0,1,1,0,1,0,0,0,3,2016,3,5,8
59531,335005,1,2016-03-05 09:23:45,2016-03-05 09:23:59,2016-03-05 09:25:43,949,MobileMessenger,Sprint,0,0,None,[],[],0,0,0,"a:1:{i:0;s:9:""depressed"";}",[depressed],[depressed],0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,2,2016,3,5,9
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
420892,700036,1,2016-12-31 23:41:31,2016-12-31 23:42:22,2016-12-31 23:46:44,952,SMS,"T-Mobile USA, Inc.",0,0,None,[],[],0,0,0,"a:2:{i:0;s:9:""depressed"";i:1;s:6:""gender"";}","[depressed, gender]","[depressed, gender]",0,0,0,0,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,2,2016,12,5,23
420895,700039,1,2016-12-31 23:44:50,2016-12-31 23:45:36,2016-12-31 23:49:39,336,MobileMessenger,Verizon Wireless,0,0,"s:19:""suicidal_capability"";",[suicidal_capability],[suicidal_capability],0,0,0,"a:2:{i:0;s:9:""depressed"";i:1;s:7:""suicide"";}","[depressed, suicide]","[depressed, suicide]",0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,1,0,0,0,3,2016,12,5,23
420900,700044,1,2016-12-31 23:52:38,2016-12-31 23:53:17,2016-12-31 23:53:49,404,SMS,AT&T Wireless,0,0,None,[],[],0,0,0,"a:1:{i:0;s:9:""depressed"";}",[depressed],[depressed],0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,2,2016,12,5,23
420902,700046,1,2016-12-31 23:56:03,2016-12-31 23:58:29,2016-12-31 23:59:19,414,SMS,US Cellular Corp.,0,0,None,[],[],0,0,0,"a:3:{i:0;s:11:""bereavement"";i:1;s:9:""depressed...","[bereavement, depressed, family]","[bereavement, depressed, family]",0,0,0,0,0,0,0,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,2,2016,12,5,23
